In [205]:
import glob
import numpy as np
import pandas as pd
from dask import delayed, compute
import dask.dataframe as dd
import pickle

data_dir = "../../data/openFDA_drug_event/"
er_dir = data_dir+'er_tables/'

try:
    os.mkdir(er_dir)
except:
    print(er_dir+" exists")

../../data/openFDA_drug_event/er_tables/ exists


## functions

In [206]:
primarykey = 'safetyreportid'

def read_file(file):
    return pd.read_csv(file,compression='gzip',index_col=0,dtype={primarykey : 'str'})

## ER tables

### report

#### report_df

In [207]:
dir_ = data_dir+'report/'
files = glob.glob(dir_+'*.csv.gzip')
results = []
for file in files:
    df = delayed(read_file)(file)
    results.append(df)
report_df = (pd.concat(compute(*results),sort=True))
report_df[primarykey] = (report_df[primarykey].astype(str))
print(report_df.columns.values)
report_df.head()

['authoritynumb' 'companynumb' 'duplicate' 'fulfillexpeditecriteria'
 'occurcountry' 'primarysource' 'primarysource.literaturereference'
 'primarysource.qualification' 'primarysource.reportercountry'
 'primarysourcecountry' 'receiptdate' 'receiptdateformat' 'receivedate'
 'receivedateformat' 'receiver' 'receiver.receiverorganization'
 'receiver.receivertype' 'reportduplicate.duplicatenumb'
 'reportduplicate.duplicatesource' 'reporttype' 'safetyreportid'
 'safetyreportversion' 'sender.senderorganization' 'sender.sendertype'
 'serious' 'seriousnesscongenitalanomali' 'seriousnessdeath'
 'seriousnessdisabling' 'seriousnesshospitalization'
 'seriousnesslifethreatening' 'seriousnessother' 'transmissiondate'
 'transmissiondateformat']


,authoritynumb,companynumb,duplicate,fulfillexpeditecriteria,occurcountry,primarysource,primarysource.literaturereference,primarysource.qualification,primarysource.reportercountry,primarysourcecountry,...,sender.sendertype,serious,seriousnesscongenitalanomali,seriousnessdeath,seriousnessdisabling,seriousnesshospitalization,seriousnesslifethreatening,seriousnessother,transmissiondate,transmissiondateformat
0,NaN,US-BAXTER-2014BAX011213,1.0,True,US,NaN,NaN,Consumer or non-health professional,US,US,...,2.0,"The adverse event resulted in death, a life th...",NaN,NaN,NaN,1.0,NaN,NaN,20141002,102
1,NaN,US-BAXTER-2014BAX012122,1.0,True,US,NaN,NaN,Physician,US,US,...,2.0,"The adverse event resulted in death, a life th...",NaN,NaN,NaN,NaN,NaN,1.0,20141002,102
2,NaN,US-BAXTER-2014BAX013311,1.0,True,US,NaN,NaN,Consumer or non-health professional,US,US,...,2.0,"The adverse event resulted in death, a life th...",NaN,NaN,NaN,1.0,NaN,NaN,20141002,102
3,NaN,RU-VIIV HEALTHCARE LIMITED-B0977587A,1.0,True,RU,NaN,NaN,Physician,RU,RU,...,2.0,"The adverse event resulted in death, a life th...",NaN,NaN,NaN,NaN,NaN,1.0,20141002,102
4,NaN,IN-BAXTER-2014BAX014013,1.0,True,IN,NaN,NaN,Physician,IN,IN,...,2.0,"The adverse event resulted in death, a life th...",NaN,NaN,NaN,NaN,NaN,1.0,20141002,102


#### report_er_df

In [15]:
columns = [primarykey,'receiptdate',
           'receivedate',
           'transmissiondate']
rename_columns = {'receiptdate' : 'mostrecent_receive_date',
                  'receivedate' : 'receive_date',
                  'transmissiondate' : 'lastupdate_date'}

report_er_df = (report_df[columns].
                rename(columns=rename_columns).
                set_index(primarykey).
                sort_index().
                reset_index().
                dropna(subset=[primarykey]).
                drop_duplicates()
               )
report_er_df = report_er_df.reindex(np.sort(report_er_df.columns),axis=1)
report_er_df[primarykey] = report_er_df[primarykey].astype(str)       
report_er_df = report_er_df.reindex(np.sort(report_er_df.columns),axis=1)
print(report_er_df.info())
report_er_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10432358 entries, 0 to 11999
Data columns (total 4 columns):
lastupdate_date            int64
mostrecent_receive_date    int64
receive_date               int64
safetyreportid             object
dtypes: int64(3), object(1)
memory usage: 398.0+ MB
None


,lastupdate_date,mostrecent_receive_date,receive_date,safetyreportid
0,20141002,20140307,20140307,9983738
1,20141002,20140317,20140317,10016373
2,20141002,20140331,20140318,10018418
3,20141002,20140318,20140318,10018999
4,20141002,20140319,20140319,10022604


In [18]:
(report_er_df.
 groupby(primarykey).
 agg(max).
 reset_index().
 dropna(subset=[primarykey])
).to_csv(er_dir+'report.csv.gz',compression='gzip',index=False)

In [ ]:
del report_er_df

### report_serious

In [19]:
columns = [primarykey,'serious',
           'seriousnesscongenitalanomali',
           'seriousnesslifethreatening',
          'seriousnessdisabling',
          'seriousnessdeath',
          'seriousnessother']
rename_columns = {           
    'seriousnesscongenitalanomali' : 'congenital_anomali',
    'seriousnesslifethreatening' : 'life_threatening',
    'seriousnessdisabling' : 'disabling',
    'seriousnessdeath' : 'death',
    'seriousnessother' : 'other'}

report_serious_er_df = (report_df[columns].
                        rename(columns=rename_columns).
                        set_index(primarykey).
                        sort_index().
                        reset_index().
                        dropna(subset=[primarykey]).
                        drop_duplicates().
                        groupby(primarykey).
                        first().
                        reset_index().
                        dropna(subset=[primarykey])
                       )
report_serious_er_df[primarykey] = report_serious_er_df[primarykey].astype(str)       
report_serious_er_df = report_serious_er_df.reindex(np.sort(report_serious_er_df.columns),axis=1)
print(report_serious_er_df.info())
report_serious_er_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10432358 entries, 0 to 11999
Data columns (total 7 columns):
safetyreportid        object
serious               object
congenital_anomali    float64
life_threatening      float64
disabling             float64
death                 float64
other                 float64
dtypes: float64(5), object(2)
memory usage: 636.7+ MB
None


,safetyreportid,serious,congenital_anomali,life_threatening,disabling,death,other
0,9983738,"The adverse event resulted in death, a life th...",NaN,NaN,NaN,NaN,NaN
1,10016373,"The adverse event resulted in death, a life th...",NaN,NaN,NaN,NaN,1.0
2,10018418,"The adverse event resulted in death, a life th...",NaN,NaN,NaN,NaN,NaN
3,10018999,"The adverse event resulted in death, a life th...",NaN,NaN,NaN,NaN,1.0
4,10022604,"The adverse event resulted in death, a life th...",NaN,NaN,NaN,NaN,1.0


In [29]:
(report_serious_er_df).to_csv(er_dir+'report_serious.csv.gz',compression='gzip',index=False)

### reporter

In [1]:
columns = [primarykey,'companynumb',
           'primarysource.qualification',
           'primarysource.reportercountry']
rename_columns = {'companynumb' : 'reporter_company',
                  'primarysource.qualification' : 'reporter_qualification',
                  'primarysource.reportercountry' : 'reporter_country'}

reporter_er_df = (report_df[columns].
                  rename(columns=rename_columns).
                  set_index(primarykey).
                  sort_index().
                  reset_index().
                  dropna(subset=[primarykey]).
                  drop_duplicates().
                  groupby(primarykey).
                  first().
                  reset_index().
                  dropna(subset=[primarykey])
                 )
reporter_er_df[primarykey] = reporter_er_df[primarykey].astype(str)  
reporter_er_df = reporter_er_df.reindex(np.sort(reporter_er_df.columns),axis=1)
print(reporter_er_df.info())
reporter_er_df.head()

NameError: name 'primarykey' is not defined

In [36]:
(reporter_er_df).to_csv(er_dir+'reporter.csv.gz',compression='gzip',index=False)

In [41]:
try:
    del df
except:
    pass
try:
    del report_df
except:
    pass
try:
    del report_serious_er_df
except:
    pass
try:
    del report_er_df
except:
    pass
try:
    del reporter_er_df
except:
    pass

### patient

#### patient_df

In [42]:
dir_ = data_dir+'patient/'
files = glob.glob(dir_+'*.csv.gzip')
results = []
for file in files:
    df = delayed(read_file)(file)
    results.append(df)
patient_df = (pd.concat(compute(*results),sort=True))
patient_df[primarykey] = (patient_df[primarykey].astype(str))
print(patient_df.columns.values)
patient_df.head()

['master_age' 'patient.patientagegroup'
 'patient.patientdeath.patientdeathdate'
 'patient.patientdeath.patientdeathdateformat' 'patient.patientonsetage'
 'patient.patientonsetageunit' 'patient.patientsex'
 'patient.patientweight' 'patient.summary.narrativeincludeclinical'
 'safetyreportid']


,master_age,patient.patientagegroup,patient.patientdeath.patientdeathdate,patient.patientdeath.patientdeathdateformat,patient.patientonsetage,patient.patientonsetageunit,patient.patientsex,patient.patientweight,patient.summary.narrativeincludeclinical,safetyreportid
0,49.0,NaN,NaN,NaN,49.0,Year,Female,NaN,NaN,6253849-7
1,25.0,NaN,NaN,NaN,25.0,Year,Female,122.4712,NaN,6253851-5
2,NaN,NaN,NaN,NaN,NaN,NaN,Female,83.9154,NaN,6253889-8
3,69.0,NaN,NaN,NaN,69.0,Year,Female,84.8226,NaN,6253908-9
4,29.0,NaN,NaN,NaN,29.0,Year,Female,72.1219,NaN,6253920-X


#### patient_er_df

In [43]:
columns = [primarykey,
              'patient.patientonsetage',
              'patient.patientonsetageunit',
              'master_age',
              'patient.patientsex',
              'patient.patientweight'
             ]
rename_columns = {
              'patient.patientonsetage' : 'patient_onsetage',
              'patient.patientonsetageunit' : 'patient_onsetageunit',
              'master_age': 'patient_custom_master_age',
              'patient.patientsex' : 'patient_sex',
              'patient.patientweight' : 'patient_weight'
}

patient_er_df = (patient_df[columns].
                 rename(columns=rename_columns).
                 set_index(primarykey).
                 sort_index().
                 reset_index().
                 dropna(subset=[primarykey]).
                 drop_duplicates().
                 groupby(primarykey).
                 first().
                 reset_index().
                 dropna(subset=[primarykey])
                )
patient_er_df = patient_er_df.reindex(np.sort(patient_er_df.columns),axis=1)
print(patient_er_df.info())
patient_er_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10434690 entries, 0 to 11999
Data columns (total 6 columns):
safetyreportid               object
patient_onsetage             float64
patient_onsetageunit         object
patient_custom_master_age    float64
patient_sex                  object
patient_weight               float64
dtypes: float64(3), object(3)
memory usage: 557.3+ MB
None


,safetyreportid,patient_onsetage,patient_onsetageunit,patient_custom_master_age,patient_sex,patient_weight
0,6253849-7,49.0,Year,49.0,Female,NaN
1,6253851-5,25.0,Year,25.0,Female,122.4712
2,6253889-8,NaN,NaN,NaN,Female,83.9154
3,6253908-9,69.0,Year,69.0,Female,84.8226
4,6253920-X,29.0,Year,29.0,Female,72.1219


In [47]:
(patient_er_df).to_csv(er_dir+'patient.csv.gz',compression='gzip',index=False)

In [48]:
del df 
del patient_df

### drug_characteristics

#### patient.drug

In [49]:
dir_ = data_dir+'patient_drug/'
files = glob.glob(dir_+'*.csv.gzip')
results = []
for file in files:
    df = delayed(read_file)(file)
    results.append(df)
patient_drug_df = (pd.concat(compute(*results),sort=True))
patient_drug_df[primarykey] = (patient_drug_df[primarykey].astype(str))
print(patient_drug_df.columns.values)
patient_drug_df.head()

/anaconda3/lib/python3.7/multiprocessing/pool.py:121: DtypeWarning: Columns (1,2,7,12,14,18) have mixed types. Specify dtype option on import or set low_memory=False.
  result = (True, func(*args, **kwds))
/anaconda3/lib/python3.7/multiprocessing/pool.py:121: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  result = (True, func(*args, **kwds))
/anaconda3/lib/python3.7/multiprocessing/pool.py:121: DtypeWarning: Columns (1,2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  result = (True, func(*args, **kwds))
/anaconda3/lib/python3.7/multiprocessing/pool.py:121: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  result = (True, func(*args, **kwds))
/anaconda3/lib/python3.7/multiprocessing/pool.py:121: DtypeWarning: Columns (2,16,20,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  result = (True, func(*args, **kwds))
/an

['actiondrug' 'activesubstance' 'drugadditional' 'drugadministrationroute'
 'drugauthorizationnumb' 'drugbatchnumb' 'drugcharacterization'
 'drugcumulativedosagenumb' 'drugcumulativedosageunit' 'drugdosageform'
 'drugdosagetext' 'drugenddate' 'drugenddateformat' 'drugindication'
 'drugintervaldosagedefinition' 'drugintervaldosageunitnumb'
 'drugrecurreadministration' 'drugrecurrence' 'drugseparatedosagenumb'
 'drugstartdate' 'drugstartdateformat' 'drugstructuredosagenumb'
 'drugstructuredosageunit' 'drugtreatmentduration'
 'drugtreatmentdurationunit' 'entry' 'medicinalproduct' 'safetyreportid']


,actiondrug,activesubstance,drugadditional,drugadministrationroute,drugauthorizationnumb,drugbatchnumb,drugcharacterization,drugcumulativedosagenumb,drugcumulativedosageunit,drugdosageform,...,drugseparatedosagenumb,drugstartdate,drugstartdateformat,drugstructuredosagenumb,drugstructuredosageunit,drugtreatmentduration,drugtreatmentdurationunit,entry,medicinalproduct,safetyreportid
0,NaN,NaN,NaN,Intravenous (not otherwise specified),125118,NaN,Suspect (the drug was considered by the report...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,ORENCIA,9111593
1,NaN,NaN,NaN,NaN,125118,NaN,Suspect (the drug was considered by the report...,NaN,NaN,NaN,...,NaN,NaN,NaN,125,NaN,NaN,NaN,0,ORENCIA,9111625
2,NaN,NaN,NaN,Intravenous (not otherwise specified),125118,NaN,Suspect (the drug was considered by the report...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,ORENCIA,9112109
3,NaN,NaN,NaN,NaN,125118,2D72776,Suspect (the drug was considered by the report...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,ORENCIA,9112332
4,NaN,NaN,NaN,NaN,125118,NaN,Suspect (the drug was considered by the report...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,ORENCIA,9112426


#### drugcharacteristics_er_df

In [60]:
columns = [primarykey,
           'medicinalproduct',
           'drugcharacterization',
           'drugadministrationroute',
           'drugindication'
          ]
rename_columns = {
              'medicinalproduct' : 'medicinal_product',
              'drugcharacterization' : 'drug_characterization',
              'drugadministrationroute': 'drug_administration',
    'drugindication' : 'drug_indication'
}

drugcharacteristics_er_df = (patient_drug_df[columns].
                             rename(columns=rename_columns).
                             set_index(primarykey).
                             sort_index().
                             reset_index().
                             drop_duplicates().
                             dropna(subset=[primarykey])
                            )
drugcharacteristics_er_df = (drugcharacteristics_er_df.
                             reindex(np.sort(drugcharacteristics_er_df.columns),axis=1))
print(drugcharacteristics_er_df.info())
drugcharacteristics_er_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32133005 entries, 0 to 29729
Data columns (total 4 columns):
safetyreportid           object
medicinal_product        object
drug_characterization    object
drug_administration      object
dtypes: object(4)
memory usage: 1.2+ GB
None


,safetyreportid,medicinal_product,drug_characterization,drug_administration
0,9111593,ORENCIA,Suspect (the drug was considered by the report...,Intravenous (not otherwise specified)
1,9111625,ORENCIA,Suspect (the drug was considered by the report...,NaN
2,9112109,ORENCIA,Suspect (the drug was considered by the report...,Intravenous (not otherwise specified)
3,9112332,ORENCIA,Suspect (the drug was considered by the report...,NaN
4,9112426,ORENCIA,Suspect (the drug was considered by the report...,NaN


In [63]:
(drugcharacteristics_er_df
).to_csv(er_dir+'drugcharacteristics.csv.gz',compression='gzip',index=False)

In [64]:
del drugcharacteristics_er_df
del patient_drug_df
del df

### drugs

#### patient.drug.openfda.rxcui_df

In [65]:
dir_ = data_dir+'patient_drug_openfda_rxcui/'
files = glob.glob(dir_+'*.csv.gzip')
results = []
for file in files:
    df = delayed(read_file)(file)
    results.append(df)
patient_drug_openfda_rxcui_df = (pd.concat(compute(*results),sort=True))
print(patient_drug_openfda_rxcui_df.columns.values)
patient_drug_openfda_rxcui_df[primarykey] = (patient_drug_openfda_rxcui_df[primarykey].
                                       astype(str))
patient_drug_openfda_rxcui_df.value = (patient_drug_openfda_rxcui_df.
                                 value.astype(int))
patient_drug_openfda_rxcui_df.head()

/anaconda3/lib/python3.7/multiprocessing/pool.py:121: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  result = (True, func(*args, **kwds))


['entry' 'key' 'safetyreportid' 'value']


,entry,key,safetyreportid,value
24,0,rxcui,14113511,643712
25,0,rxcui,14113511,643714
26,0,rxcui,14113511,1429367
27,0,rxcui,14113511,643720
28,0,rxcui,14113511,643722


#### drugs_er_df

In [66]:
columns = [primarykey,
              'value'
             ]
rename_columns = {
              'value' : 'rxcui'
}

drugs_er_df = (patient_drug_openfda_rxcui_df[columns].
               rename(columns=rename_columns).
               set_index(primarykey).
               sort_index().
               reset_index().
               drop_duplicates().
               dropna(subset=[primarykey])
              )
drugs_er_df = drugs_er_df.reindex(np.sort(drugs_er_df.columns),axis=1)
print(drugs_er_df.info())
drugs_er_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247682846 entries, 0 to 292994179
Data columns (total 2 columns):
safetyreportid    object
rxcui             int64
dtypes: int64(1), object(1)
memory usage: 5.5+ GB
None


,safetyreportid,rxcui
0,10003300,904952
1,10003300,904932
2,10003300,904934
3,10003300,904954
4,10003301,206917


In [70]:
drugs_er_df['rxcui'] = drugs_er_df['rxcui'].astype(int)

In [71]:
drugs_er_df[primarykey] = drugs_er_df[primarykey].astype(str)

In [72]:
(drugs_er_df).to_csv(er_dir+'drugs.csv.gz',compression='gzip',index=False)

In [73]:
del patient_drug_openfda_rxcui_df
del drugs_er_df
del df

### reactions

#### patient.reaction_df

In [74]:
dir_ = data_dir+'patient_reaction/'
files = glob.glob(dir_+'*.csv.gzip')
results = []
for file in files:
    df = delayed(read_file)(file)
    results.append(df)
patient_reaction_df = (pd.concat(compute(*results),sort=True))
patient_reaction_df[primarykey] = (patient_reaction_df[primarykey].astype(str))
print(patient_reaction_df.columns.values)
patient_reaction_df.head()

['entry' 'reactionmeddrapt' 'reactionmeddraversionpt' 'reactionoutcome'
 'safetyreportid']


,entry,reactionmeddrapt,reactionmeddraversionpt,reactionoutcome,safetyreportid
0,0,Dyspnoea,21.0,Unknown,15386347
1,1,Anaemia,21.0,Unknown,15386347
2,2,Decreased appetite,21.0,Unknown,15386347
3,3,Chest discomfort,21.0,Unknown,15386347
4,4,Dizziness,21.0,Unknown,15386347


#### patient_reaction_er_df

In [75]:
columns = [primarykey,
              'reactionmeddrapt',
           'reactionoutcome'
             ]
rename_columns = {
              'reactionmeddrapt' : 'reaction_meddrapt',
    'reactionoutcome' : 'reaction_outcome'
}

reactions_er_df = (patient_reaction_df[columns].
                   rename(columns=rename_columns).
                   set_index(primarykey).
                   sort_index().
                   reset_index().
                   dropna(subset=[primarykey]).
                   drop_duplicates()
                  )
reactions_er_df[primarykey] = reactions_er_df[primarykey].astype(str)
reactions_er_df = reactions_er_df.reindex(np.sort(reactions_er_df.columns),axis=1)
print(reactions_er_df.info())
reactions_er_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19209600 entries, 6 to 31511719
Data columns (total 3 columns):
safetyreportid       object
reaction_meddrapt    object
reaction_outcome     object
dtypes: object(3)
memory usage: 586.2+ MB
None


,safetyreportid,reaction_meddrapt,reaction_outcome
6,10003302,Drug ineffective,Unknown
7,10003304,Drug hypersensitivity,Unknown
8,10003305,Drug hypersensitivity,Unknown
9,10003306,Cough,Unknown
10,10003306,Throat irritation,Unknown


In [76]:
(reactions_er_df).to_csv(er_dir+'reactions.csv.gz',compression='gzip',index=False)

In [77]:
del patient_reaction_df
del reactions_er_df
del df

### omop tables for joining

In [110]:
concept = (pd.read_csv('../../vocabulary_SNOMED_MEDDRA_RxNorm_ATC/CONCEPT.csv',sep='\t',
                      dtype={
                          'concept_id' : 'int'
                      }))
concept.head()

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,45956931,Self-emulsifying glyceryl monostearate,Observation,SNOMED,Substance,S,3578611000001105,19700101,20991231,NaN
1,45956935,Sibutramine hydrochloride,Observation,SNOMED,Substance,S,3579011000001108,19700101,20991231,NaN
2,45956943,Yellow soft paraffin,Observation,SNOMED,Substance,S,3330611000001108,19700101,20991231,NaN
3,45956945,Magnesium oxide light,Observation,SNOMED,Substance,S,3565611000001107,19700101,20991231,NaN
4,45956950,Lanolin oil,Observation,SNOMED,Substance,S,3563811000001107,19700101,20991231,NaN


In [111]:
concept_relationship = (pd.
                        read_csv('../../vocabulary_SNOMED_MEDDRA_RxNorm_ATC/'+
                                 'CONCEPT_RELATIONSHIP.csv',sep='\t',
                                dtype={
                                    'concept_id_1' : 'int',
                                    'concept_id_2' : 'int'
                                }))
concept_relationship.head()

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
0,36237052,36217216,Has dose form group,20160801,20991231,NaN
1,19125288,36217755,RxNorm is a,20160801,20991231,NaN
2,36214341,36217216,Has dose form group,20160801,20991231,NaN
3,19103333,36212953,RxNorm is a,20160801,20991231,NaN
4,36215077,36217214,Has dose form group,20160801,20991231,NaN


### standard_drugs

In [5]:
drugs = (pd.read_csv(
    er_dir+'drugs.csv.gz',
    compression='gzip',
    dtype={
        'safetyreportid' : 'str'
    }
)
        )

In [6]:
drugs['rxcui'] = drugs['rxcui'].astype(int)

In [7]:
urxcuis = drugs['rxcui'].unique()

In [8]:
print(len(urxcuis))
urxcuis[:5]

707


array([ 904954,  904934,  904952,  904932, 1300267])

In [9]:
rxnorm_concept = concept.query('vocabulary_id=="RxNorm"')

In [10]:
concept_codes = rxnorm_concept['concept_code'].astype(int).unique()
print(len(concept_codes))
print(len(urxcuis))

intersect = np.intersect1d(concept_codes,urxcuis)

print(len(intersect))
print(len(intersect)/len(urxcuis))

285233
707
696
0.9844413012729845


In [11]:
del urxcuis
del concept_codes

In [12]:
rxnorm_concept = concept.query('vocabulary_id=="RxNorm"')

rxnorm_concept_ids = (rxnorm_concept.
                      query('concept_code in @intersect')['concept_id'].
                      astype(int).
                      unique()
                     )
all_rxnorm_concept_ids = (rxnorm_concept['concept_id'].
                          unique()
                         )

r = (concept_relationship.
     copy().
     loc[:,['concept_id_1','concept_id_2','relationship_id']].
     drop_duplicates()
    )
c = rxnorm_concept.copy()
c['concept_id'] = c['concept_id'].astype(int)
c['concept_code'] = c['concept_code'].astype(int)

joined = (drugs.
          set_index('rxcui').
          join(
              c. 
              query('vocabulary_id=="RxNorm"').
              loc[:,['concept_id','concept_code','concept_name','concept_class_id']].
              drop_duplicates().
              set_index('concept_code')
          ).
          dropna().
          rename_axis('RxNorm_concept_code').
          reset_index().
          rename(
              columns={
                  'concept_class_id' : 'RxNorm_concept_class_id',
                  'concept_name' : 'RxNorm_concept_name',
                  'concept_id' : 'RxNorm_concept_id'
              }
          ).
          dropna(subset=['RxNorm_concept_id']).
          drop_duplicates()
         )
joined = (joined.
          reindex(np.sort(joined.columns),axis=1)
         )
print(joined.shape)
print(joined.head())

(987, 5)
  RxNorm_concept_class_id  RxNorm_concept_code  RxNorm_concept_id  \
0            Branded Drug               104375         19003829.0   
1            Branded Drug               104376         19003830.0   
2            Branded Drug               104377         19003851.0   
3            Branded Drug               104378         19003852.0   
4            Branded Drug               104490         19004081.0   

                       RxNorm_concept_name  safetyreportid  
0  Lisinopril 2.5 MG Oral Tablet [Zestril]        10003305  
1    Lisinopril 5 MG Oral Tablet [Zestril]        10003305  
2   Lisinopril 10 MG Oral Tablet [Zestril]        10003305  
3   Lisinopril 20 MG Oral Tablet [Zestril]        10003305  
4    Simvastatin 10 MG Oral Tablet [Zocor]        10003310  


In [13]:
len(np.intersect1d(joined.RxNorm_concept_code.unique(),intersect))/len(intersect)

1.0

In [14]:
ids = joined.RxNorm_concept_id.dropna().astype(int).unique()

In [117]:
pickle.dump(
    ids,
    open('../../data/all_openFDA_rxnorm_concept_ids.pkl','wb')
)

In [ ]:
(joined.to_csv(er_dir+'standard_drugs.csv.gz',compression='gzip',index=False))

In [15]:
del joined

### standard_reactions

In [25]:
patient_reaction_df = (pd.read_csv(
    er_dir+'reactions.csv.gz',
    compression='gzip',
                               dtype={
                                   'safetyreportid' : 'str'
                               }
                              ))
all_reports = patient_reaction_df.safetyreportid.unique()
print(patient_reaction_df.columns)
print(patient_reaction_df.safetyreportid.nunique())
print(patient_reaction_df.reaction_meddrapt.nunique())

Index(['safetyreportid', 'reaction_meddrapt', 'reaction_outcome'], dtype='object')
390
442


In [18]:
patient_reaction_df.head()

,safetyreportid,reaction_meddrapt,reaction_outcome
0,10003300,Headache,NaN
1,10003300,Arthralgia,NaN
2,10003300,Diarrhoea,NaN
3,10003300,Vomiting,NaN
4,10003301,Dyspepsia,NaN


In [19]:
meddra_concept = concept.query('vocabulary_id=="MedDRA"')
meddra_concept.head()

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
397065,46276554,Vascular compression therapy,Procedure,MedDRA,PT,C,10076482,20150901,20991231,NaN
397066,46276555,Neurological rehabilitation,Procedure,MedDRA,PT,C,10076483,20150901,20991231,NaN
397067,46276556,Synovial cyst removal,Procedure,MedDRA,PT,C,10076485,20150901,20991231,NaN
397068,46276557,Gluten sensitivity,Condition,MedDRA,PT,C,10076493,20150901,20991231,NaN
397069,46276558,Meconium aspiration syndrome,Condition,MedDRA,PT,C,10076496,20150901,20991231,NaN


In [20]:
reactions = patient_reaction_df.reaction_meddrapt.copy().astype(str).str.title().unique()
print(len(reactions))
concept_names = meddra_concept.concept_name.astype(str).str.title().unique()
print(len(concept_names))

intersect_title = np.intersect1d(reactions,concept_names)
print(len(intersect_title))

print(len(intersect_title)/len(reactions))

442
78610
441
0.997737556561086


In [21]:
patient_reaction_df['reaction_meddrapt'] = (patient_reaction_df['reaction_meddrapt'].
                                            astype(str).
                                            str.
                                            title())
meddra_concept['concept_name'] = (meddra_concept['concept_name'].
                                  astype(str).
                                  str.
                                  title())
print(patient_reaction_df.shape[0])

joined = ((patient_reaction_df.
  set_index('reaction_meddrapt').
  join(
      meddra_concept.
      query('concept_class_id=="PT"').
      loc[:,['concept_id','concept_name','concept_code','concept_class_id']].
      drop_duplicates().
      set_index('concept_name')
  ).
           rename(columns={'concept_id' : 'MedDRA_concept_id',
                          'concept_code' : 'MedDRA_concept_code',
                          'concept_class_id' : 'MedDRA_concept_class_id'}).
           drop_duplicates()
 )
).rename_axis('MedDRA_concept_name').reset_index()
joined = joined.reindex(np.sort(joined.columns),axis=1)
print(joined.shape[0])
print(joined.head())

1000
1000
    MedDRA_concept_name  safetyreportid            reaction_outcome  \
0                Abasia        10003542  Not recovered/not resolved   
1  Abdominal Discomfort        10003414                     Unknown   
2  Abdominal Discomfort        10003677                     Unknown   
3  Abdominal Distension        10003409                     Unknown   
4  Abdominal Distension        10003427          Recovered/resolved   

   MedDRA_concept_id MedDRA_concept_code MedDRA_concept_class_id  
0         35809100.0            10049460                      PT  
1         35708164.0            10000059                      PT  
2         35708164.0            10000059                      PT  
3         35708151.0            10000060                      PT  
4         35708151.0            10000060                      PT  


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
del meddra_concept
del patient_reaction_df

In [23]:
joined_notnull = joined[joined.MedDRA_concept_id.notnull()]
print(joined_notnull.shape[0])
joined_notnull['MedDRA_concept_id'] = joined_notnull['MedDRA_concept_id'].astype(int)
print(joined_notnull.head())

990
    MedDRA_concept_name  safetyreportid            reaction_outcome  \
0                Abasia        10003542  Not recovered/not resolved   
1  Abdominal Discomfort        10003414                     Unknown   
2  Abdominal Discomfort        10003677                     Unknown   
3  Abdominal Distension        10003409                     Unknown   
4  Abdominal Distension        10003427          Recovered/resolved   

   MedDRA_concept_id MedDRA_concept_code MedDRA_concept_class_id  
0           35809100            10049460                      PT  
1           35708164            10000059                      PT  
2           35708164            10000059                      PT  
3           35708151            10000060                      PT  
4           35708151            10000060                      PT  


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
print(
    len(
        np.intersect1d(
            all_reports,
            joined_notnull.safetyreportid.astype(str).unique()
        )
    )/len(all_reports)
)

0.9923076923076923


In [24]:
print(joined_notnull.MedDRA_concept_class_id.value_counts())
print(joined_notnull.safetyreportid.nunique())
print(joined_notnull.MedDRA_concept_id.nunique())

PT    990
Name: MedDRA_concept_class_id, dtype: int64
387
434


In [15]:
pickle.dump(
    joined_notnull.MedDRA_concept_id.astype(int).unique,
    open('../../data/all_openFDA_meddra_concept_ids.pkl','wb')
)

In [18]:
(joined_notnull.to_csv(er_dir+'standard_reactions.csv.gz',compression='gzip',index=False))

In [16]:
del joined_notnull

In [19]:
del joined

### standard_drugs_atc

In [74]:
standard_drugs = (pd.read_csv(
    er_dir+'standard_drugs.csv.gz',
    compression='gzip',
    dtype={
        'safetyreportid' : 'str'
    }
))

In [75]:
all_reports = standard_drugs.safetyreportid.unique()
len(all_reports)

1000

In [76]:
standard_drugs.RxNorm_concept_id = standard_drugs.RxNorm_concept_id.astype(int)

In [77]:
standard_drugs.head()

,RxNorm_concept_class_id,RxNorm_concept_code,RxNorm_concept_id,RxNorm_concept_name,safetyreportid
0,Clinical Drug,91349,1776544,Hydrogen Peroxide 30 MG/ML Topical Solution,10024478
1,Clinical Drug,91349,1776544,Hydrogen Peroxide 30 MG/ML Topical Solution,10037760
2,Clinical Drug,91349,1776544,Hydrogen Peroxide 30 MG/ML Topical Solution,10162316
3,Clinical Drug,91349,1776544,Hydrogen Peroxide 30 MG/ML Topical Solution,10235601
4,Clinical Drug,91349,1776544,Hydrogen Peroxide 30 MG/ML Topical Solution,10284414


In [78]:
rxnorm_concept = concept.query('vocabulary_id=="RxNorm"')
rxnorm_concept_ids = rxnorm_concept['concept_id'].unique()

In [79]:
openfda_concept_ids = standard_drugs.RxNorm_concept_id.dropna().astype(int).unique()

In [80]:
atc_concept = concept.query('vocabulary_id=="ATC" & concept_class_id=="ATC 5th"')

r = (concept_relationship.
     copy().
     loc[:,['concept_id_1','concept_id_2','relationship_id']].
     drop_duplicates()
    )
                            
r['concept_id_1'] = r['concept_id_1'].astype(int)
r['concept_id_2'] = r['concept_id_2'].astype(int)
ac = atc_concept.copy()
ac['concept_id'] = ac['concept_id'].astype(int)
atc_concept_ids = ac['concept_id'].unique()
rc = rxnorm_concept.copy()
rc['concept_id'] = rc['concept_id'].astype(int)
rxnorm_concept_ids = rc['concept_id'].unique()

In [81]:
rxnorm_to_atc_relationships = (r.
                         query('concept_id_1 in @openfda_concept_ids & '\
                               'concept_id_2 in @atc_concept_ids').
                         set_index('concept_id_1').
                         join(
                             rc. # standard concepts for 1
                             loc[:,['concept_id','concept_code',
                                    'concept_name','concept_class_id']].
                             drop_duplicates().
                             set_index('concept_id')
                         ).
                         rename_axis('RxNorm_concept_id').
                         reset_index().
                         dropna().
                         rename(
                             columns={
                                 'concept_code' : 'RxNorm_concept_code',
                                 'concept_class_id' : 'RxNorm_concept_class_id',
                                 'concept_name' : 'RxNorm_concept_name',
                                 'concept_id_2' : 'ATC_concept_id',
                             }
                         ).
                         set_index('ATC_concept_id').
                         join(
                             ac. # standard concepts for 2
                             loc[:,['concept_id','concept_code',
                                    'concept_name','concept_class_id']].
                             drop_duplicates().
                             set_index('concept_id')
                         ).
                         dropna().
                         rename_axis('ATC_concept_id').
                         reset_index().
                         rename(
                             columns={
                                 'concept_code' : 'ATC_concept_code',
                                 'concept_class_id' : 'ATC_concept_class_id',
                                 'concept_name' : 'ATC_concept_name'
                             }
                         )
                        )
rxnorm_to_atc_relationships.RxNorm_concept_id = \
(rxnorm_to_atc_relationships.RxNorm_concept_id.
astype(int))
rxnorm_to_atc_relationships.ATC_concept_id = \
(rxnorm_to_atc_relationships.ATC_concept_id.
astype(int))

rxnorm_to_atc_relationships = (rxnorm_to_atc_relationships.
                            reindex(np.sort(rxnorm_to_atc_relationships.columns),axis=1)
                           )
print(rxnorm_to_atc_relationships.shape)
print(rxnorm_to_atc_relationships.head())

(2, 9)
  ATC_concept_class_id ATC_concept_code  ATC_concept_id   ATC_concept_name  \
0              ATC 5th          A01AB02        21600012  hydrogen peroxide   
1              ATC 5th          A14AA05        21600921       oxymetholone   

  RxNorm_concept_class_id RxNorm_concept_code  RxNorm_concept_id  \
0           Clinical Drug               91349            1776544   
1            Branded Drug               91792            1525764   

                           RxNorm_concept_name      relationship_id  
0  Hydrogen Peroxide 30 MG/ML Topical Solution  Drug has drug class  
1  Oxymetholone 50 MG Oral Tablet [Anadrol-50]         RxNorm - ATC  


In [82]:
rxnorm_to_atc_relationships.ATC_concept_class_id.value_counts()

ATC 5th    2
Name: ATC_concept_class_id, dtype: int64

In [83]:
del r
del ac
del rc

In [85]:
standard_drugs_atc = (standard_drugs.
                      loc[:,['RxNorm_concept_id','safetyreportid']].
                      drop_duplicates().
                      set_index('RxNorm_concept_id').
                      join(rxnorm_to_atc_relationships.
                           set_index('RxNorm_concept_id')
                          ).
                      drop_duplicates().
                      reset_index(drop=True).
                      drop(['RxNorm_concept_code','RxNorm_concept_name',
                            'RxNorm_concept_class_id','relationship_id'],axis=1).
                      dropna(subset=['ATC_concept_id']).
                      drop_duplicates()
                     )

standard_drugs_atc = \
standard_drugs_atc.reindex(np.sort(standard_drugs_atc.columns),axis=1)
standard_drugs_atc.ATC_concept_id = standard_drugs_atc.ATC_concept_id.astype(int)
print(len(
    np.intersect1d(all_reports,
                   standard_drugs_atc.safetyreportid.unique()
                  )
)/len(all_reports))
print(standard_drugs_atc.shape)
print(standard_drugs_atc.info())
print(standard_drugs_atc.head())

0.174
(174, 5)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 174 entries, 826 to 999
Data columns (total 5 columns):
ATC_concept_class_id    174 non-null object
ATC_concept_code        174 non-null object
ATC_concept_id          174 non-null int64
ATC_concept_name        174 non-null object
safetyreportid          174 non-null object
dtypes: int64(1), object(4)
memory usage: 8.2+ KB
None
    ATC_concept_class_id ATC_concept_code  ATC_concept_id ATC_concept_name  \
826              ATC 5th          A14AA05        21600921     oxymetholone   
827              ATC 5th          A14AA05        21600921     oxymetholone   
828              ATC 5th          A14AA05        21600921     oxymetholone   
829              ATC 5th          A14AA05        21600921     oxymetholone   
830              ATC 5th          A14AA05        21600921     oxymetholone   

    safetyreportid  
826       10062556  
827       10404396  
828       10404398  
829       10486816  
830       11069338  


In [37]:
del standard_drugs
del rxnorm_to_atc_relationships

In [38]:
standard_drugs_atc.to_csv(er_dir+'standard_drugs_atc.csv.gz',compression='gzip',index=False)

In [39]:
del standard_drugs_atc

### standard_drugs_rxnorm_ingredients

https://www.nlm.nih.gov/research/umls/rxnorm/overview.html

In [152]:
all_openFDA_rxnorm_concept_ids = pickle.load(
    open('../../data/all_openFDA_rxnorm_concept_ids.pkl','rb')
)

In [153]:
all_openFDA_rxnorm_concept_ids

array([1776544, 1525764,  905343, ..., 1356436, 1356428, 1356430])

In [154]:
all_rxnorm_concept_ids = (concept.
                          query('vocabulary_id=="RxNorm"').
                          concept_id.
                          astype(int).
                          unique()
                         )

In [155]:
r = (concept_relationship.
     loc[:,['concept_id_1','concept_id_2','relationship_id']].
    drop_duplicates().
    dropna().
     copy()
    )
r.concept_id_1 = r.concept_id_1.astype(int)
r.concept_id_2 = r.concept_id_2.astype(int)

In [156]:
c = (concept.
    query('vocabulary_id=="RxNorm" & standard_concept=="S"').
    loc[:,['concept_id','concept_code',
          'concept_class_id','concept_name']].
    drop_duplicates().
    dropna().
     copy()
    )
c.concept_id = c.concept_id.astype(int).copy()

In [157]:
all_rxnorm_concept_ids = concept.query('vocabulary_id=="RxNorm"').concept_id.astype(int).unique()
rxnorm_relationships = (r.
 query('concept_id_1 in @all_rxnorm_concept_ids & '+
       'concept_id_2 in @all_rxnorm_concept_ids').
 relationship_id.
 value_counts()
)
rxnorm_relationships

RxNorm inverse is a     208496
RxNorm is a             208496
Mapped from             192919
Maps to                 192919
Brand name of           109992
Has brand name          109992
RxNorm dose form of     100994
RxNorm has dose form    100994
Tradename of             99814
Has tradename            99814
RxNorm has ing           84302
RxNorm ing of            84302
Consists of              79464
Constitutes              79464
Concept replaced by      40711
Concept replaces         40711
Dose form group of       38011
Has dose form group      38011
Has precise ing          12614
Precise ing of           12614
Quantified form of        5797
Has quantified form       5797
Has form                  2843
Form of                   2843
Contained in              2347
Contains                  2347
Has component                1
Component of                 1
Name: relationship_id, dtype: int64

In [158]:
first_second_relations = (r.
                          query('concept_id_1 in @all_openFDA_rxnorm_concept_ids').
                          set_index('concept_id_1').
                          join(c.
                               set_index('concept_id')).
                          rename(
                              columns={
                                  'concept_id_1' : 'RxNorm_concept_id_1',
                                  'concept_code' : 'RxNorm_concept_code_1',
                                  'concept_class_id' : 'RxNorm_concept_class_id_1',
                                  'concept_name' : 'RxNorm_concept_name_1'
                              }
                          ).
                          rename_axis('RxNorm_concept_id_1').
                          reset_index().
                          set_index('concept_id_2').
                          join(c.
                               set_index('concept_id')
                              ).
                          rename(
                              columns={'concept_id_2' : 'RxNorm_concept_id_2',
                                       'concept_code' : 'RxNorm_concept_code_2',
                                       'concept_class_id' : 'RxNorm_concept_class_id_2',
                                       'concept_name' : 'RxNorm_concept_name_2',
                                       'relationship_id' :'relationship_id_12'
                                      }
                          ).
                          rename_axis('RxNorm_concept_id_2').
                          reset_index().
                          dropna().
                          drop_duplicates()
                         )
first_second_relations = first_second_relations[
    first_second_relations.RxNorm_concept_id_1!=first_second_relations.RxNorm_concept_id_2
]
print(first_second_relations.shape)
first_second_relations = (first_second_relations.
                          reindex(np.sort(first_second_relations.columns),
                                  axis=1)
                         )
print(first_second_relations.head())

(54919, 9)
   RxNorm_concept_class_id_1 RxNorm_concept_class_id_2 RxNorm_concept_code_1  \
16             Clinical Drug        Clinical Drug Comp                313362   
17             Clinical Drug        Clinical Drug Comp                313366   
18             Clinical Drug        Clinical Drug Comp                313361   
32             Clinical Drug        Clinical Drug Comp                197928   
33             Clinical Drug        Clinical Drug Comp                197929   

   RxNorm_concept_code_2  RxNorm_concept_id_1  RxNorm_concept_id_2  \
16                328731               700494               700501   
17                328732               700498               700522   
18                328733               700493               700523   
32                316243               702867               702994   
33                316244               702868               702995   

             RxNorm_concept_name_1 RxNorm_concept_name_2 relationship_id_12  
16   Thio

In [159]:
(first_second_relations.loc[:,['RxNorm_concept_class_id_1','RxNorm_concept_class_id_2']].
groupby(['RxNorm_concept_class_id_1','RxNorm_concept_class_id_2']).
 count()
)

Empty DataFrame
Columns: []
Index: [(Branded Drug, Branded Drug Comp), (Branded Drug, Branded Drug Form), (Branded Drug, Branded Pack), (Branded Drug, Clinical Drug), (Branded Drug, Quant Branded Drug), (Branded Pack, Branded Drug), (Branded Pack, Clinical Drug), (Branded Pack, Clinical Pack), (Branded Pack, Quant Branded Drug), (Branded Pack, Quant Clinical Drug), (Clinical Drug, Branded Drug), (Clinical Drug, Branded Pack), (Clinical Drug, Clinical Drug Comp), (Clinical Drug, Clinical Drug Form), (Clinical Drug, Clinical Pack), (Clinical Drug, Quant Clinical Drug), (Clinical Pack, Branded Pack), (Clinical Pack, Clinical Drug), (Clinical Pack, Quant Clinical Drug), (Quant Branded Drug, Branded Drug), (Quant Branded Drug, Branded Pack), (Quant Branded Drug, Quant Clinical Drug), (Quant Clinical Drug, Branded Pack), (Quant Clinical Drug, Clinical Drug), (Quant Clinical Drug, Clinical Pack), (Quant Clinical Drug, Quant Branded Drug)]

In [160]:
ids = first_second_relations.RxNorm_concept_id_2.astype(int).unique()

second_third_relations = (r.
                          query('concept_id_1 in @ids').
                          set_index('concept_id_1').
                          join(c.
                               set_index('concept_id')).
                          rename(
                              columns={
                                  'concept_id_1' : 'RxNorm_concept_id_2',
                                  'concept_code' : 'RxNorm_concept_code_2',
                                  'concept_class_id' : 'RxNorm_concept_class_id_2',
                                  'concept_name' : 'RxNorm_concept_name_2'
                              }
                          ).
                          rename_axis('RxNorm_concept_id_2').
                          reset_index().
                          set_index('concept_id_2').
                          join(c.
                               set_index('concept_id')
                              ).
                          rename(
                              columns={'concept_id_2' : 'RxNorm_concept_id_3',
                                       'concept_code' : 'RxNorm_concept_code_3',
                                       'concept_class_id' : 'RxNorm_concept_class_id_3',
                                       'concept_name' : 'RxNorm_concept_name_3',
                                       'relationship_id' :'relationship_id_23'
                                      }
                          ).
                          rename_axis('RxNorm_concept_id_3').
                          reset_index().
                          dropna().
                          drop_duplicates()
                         )
second_third_relations = second_third_relations[
    second_third_relations.RxNorm_concept_id_2!=second_third_relations.RxNorm_concept_id_3
]
print(second_third_relations.shape)
second_third_relations = (second_third_relations.
                          reindex(np.sort(second_third_relations.columns),
                                  axis=1)
                         )
print(second_third_relations.head())

(167146, 9)
  RxNorm_concept_class_id_2 RxNorm_concept_class_id_3 RxNorm_concept_code_2  \
2             Clinical Drug        Clinical Drug Comp               1809417   
3        Clinical Drug Comp                Ingredient                328704   
4        Clinical Drug Comp                Ingredient                328706   
5        Clinical Drug Comp                Ingredient                328708   
6        Clinical Drug Comp                Ingredient                328710   

  RxNorm_concept_code_3  RxNorm_concept_id_2  RxNorm_concept_id_3  \
2                343818             40220527               551822   
3                 10502             19083127               700299   
4                 10502             19083129               700299   
5                 10502             19083130               700299   
6                 10502             19083151               700299   

                         RxNorm_concept_name_2  \
2  Immunoglobulin G, Human 100 MG/ML Injection  

In [161]:
(second_third_relations.loc[:,['RxNorm_concept_class_id_2','RxNorm_concept_class_id_3']].
groupby(['RxNorm_concept_class_id_2','RxNorm_concept_class_id_3']).
 count()
)

Empty DataFrame
Columns: []
Index: [(Branded Drug, Branded Drug Comp), (Branded Drug, Branded Drug Form), (Branded Drug, Branded Pack), (Branded Drug, Clinical Drug), (Branded Drug, Quant Branded Drug), (Branded Drug Comp, Branded Drug), (Branded Drug Comp, Clinical Drug Comp), (Branded Drug Form, Branded Drug), (Branded Drug Form, Clinical Drug Form), (Branded Pack, Branded Drug), (Branded Pack, Clinical Drug), (Branded Pack, Clinical Pack), (Branded Pack, Quant Branded Drug), (Branded Pack, Quant Clinical Drug), (Clinical Drug, Branded Drug), (Clinical Drug, Branded Pack), (Clinical Drug, Clinical Drug Comp), (Clinical Drug, Clinical Drug Form), (Clinical Drug, Clinical Pack), (Clinical Drug, Quant Clinical Drug), (Clinical Drug Comp, Branded Drug Comp), (Clinical Drug Comp, Clinical Drug), (Clinical Drug Comp, Ingredient), (Clinical Drug Form, Branded Drug Form), (Clinical Drug Form, Clinical Drug), (Clinical Drug Form, Ingredient), (Clinical Pack, Branded Pack), (Clinical Pack, Clinical Drug), (Clinical Pack, Quant Clinical Drug), (Quant Branded Drug, Branded Drug), (Quant Branded Drug, Branded Pack), (Quant Branded Drug, Quant Clinical Drug), (Quant Clinical Drug, Branded Pack), (Quant Clinical Drug, Clinical Drug), (Quant Clinical Drug, Clinical Pack), (Quant Clinical Drug, Quant Branded Drug)]

In [162]:
ids = second_third_relations.RxNorm_concept_id_3.astype(int).unique()

third_fourth_relations = (r.
                          query('concept_id_1 in @ids').
                          set_index('concept_id_1').
                          join(c.
                               set_index('concept_id')).
                          rename(
                              columns={
                                  'concept_id_1' : 'RxNorm_concept_id_3',
                                  'concept_code' : 'RxNorm_concept_code_3',
                                  'concept_class_id' : 'RxNorm_concept_class_id_3',
                                  'concept_name' : 'RxNorm_concept_name_3'
                              }
                          ).
                          rename_axis('RxNorm_concept_id_3').
                          reset_index().
                          set_index('concept_id_2').
                          join(c.
                               set_index('concept_id')
                              ).
                          rename(
                              columns={'concept_id_2' : 'RxNorm_concept_id_4',
                                       'concept_code' : 'RxNorm_concept_code_4',
                                       'concept_class_id' : 'RxNorm_concept_class_id_4',
                                       'concept_name' : 'RxNorm_concept_name_4',
                                       'relationship_id' :'relationship_id_34'
                                      }
                          ).
                          rename_axis('RxNorm_concept_id_4').
                          reset_index().
                          dropna().
                          drop_duplicates()
                         )
third_fourth_relations = third_fourth_relations[
    third_fourth_relations.RxNorm_concept_id_3!=third_fourth_relations.RxNorm_concept_id_4
]
print(third_fourth_relations.shape)
third_fourth_relations = (third_fourth_relations.
                          reindex(np.sort(third_fourth_relations.columns),
                                  axis=1)
                         )
print(third_fourth_relations.head())

(268568, 9)
   RxNorm_concept_class_id_3 RxNorm_concept_class_id_4 RxNorm_concept_code_3  \
0         Clinical Drug Comp                Ingredient                705903   
1         Clinical Drug Form                Ingredient               1658124   
13        Clinical Drug Comp                Ingredient               1790372   
14        Clinical Drug Form                Ingredient               1790373   
15        Clinical Drug Comp                Ingredient               1790505   

   RxNorm_concept_code_4  RxNorm_concept_id_3  RxNorm_concept_id_4  \
0                  26744             19126484               501343   
1                  26744             46275211               501343   
13                 35465             42629480               535714   
14                 35465             42629481               535714   
15                 35465             42629496               535714   

                        RxNorm_concept_name_3        RxNorm_concept_name_4  \
0      h

In [163]:
(third_fourth_relations.loc[:,['RxNorm_concept_class_id_3','RxNorm_concept_class_id_4']].
groupby(['RxNorm_concept_class_id_3','RxNorm_concept_class_id_4']).
 count()
)

Empty DataFrame
Columns: []
Index: [(Branded Drug, Branded Drug Comp), (Branded Drug, Branded Drug Form), (Branded Drug, Branded Pack), (Branded Drug, Clinical Drug), (Branded Drug, Quant Branded Drug), (Branded Drug Comp, Branded Drug), (Branded Drug Comp, Clinical Drug Comp), (Branded Drug Form, Branded Drug), (Branded Drug Form, Clinical Drug Form), (Branded Pack, Branded Drug), (Branded Pack, Clinical Drug), (Branded Pack, Clinical Pack), (Branded Pack, Quant Branded Drug), (Branded Pack, Quant Clinical Drug), (Clinical Drug, Branded Drug), (Clinical Drug, Branded Pack), (Clinical Drug, Clinical Drug Comp), (Clinical Drug, Clinical Drug Form), (Clinical Drug, Clinical Pack), (Clinical Drug, Quant Clinical Drug), (Clinical Drug Comp, Branded Drug Comp), (Clinical Drug Comp, Clinical Drug), (Clinical Drug Comp, Ingredient), (Clinical Drug Form, Branded Drug Form), (Clinical Drug Form, Clinical Drug), (Clinical Drug Form, Ingredient), (Clinical Pack, Branded Pack), (Clinical Pack, Clinical Drug), (Clinical Pack, Quant Clinical Drug), (Ingredient, Clinical Drug Comp), (Ingredient, Clinical Drug Form), (Quant Branded Drug, Branded Drug), (Quant Branded Drug, Branded Pack), (Quant Branded Drug, Quant Clinical Drug), (Quant Clinical Drug, Branded Pack), (Quant Clinical Drug, Clinical Drug), (Quant Clinical Drug, Clinical Pack), (Quant Clinical Drug, Quant Branded Drug)]

In [164]:
ids = third_fourth_relations.RxNorm_concept_id_4.astype(int).unique()

fourth_fifth_relations = (r.
                          query('concept_id_1 in @ids').
                          set_index('concept_id_1').
                          join(c.
                               set_index('concept_id')).
                          rename(
                              columns={
                                  'concept_id_1' : 'RxNorm_concept_id_4',
                                  'concept_code' : 'RxNorm_concept_code_4',
                                  'concept_class_id' : 'RxNorm_concept_class_id_4',
                                  'concept_name' : 'RxNorm_concept_name_4'
                              }
                          ).
                          rename_axis('RxNorm_concept_id_4').
                          reset_index().
                          set_index('concept_id_2').
                          join(c.
                               set_index('concept_id')
                              ).
                          rename(
                              columns={'concept_id_2' : 'RxNorm_concept_id_5',
                                       'concept_code' : 'RxNorm_concept_code_5',
                                       'concept_class_id' : 'RxNorm_concept_class_id_5',
                                       'concept_name' : 'RxNorm_concept_name_5',
                                       'relationship_id' :'relationship_id_45'
                                      }
                          ).
                          rename_axis('RxNorm_concept_id_5').
                          reset_index().
                          dropna().
                          drop_duplicates()
                         )
fourth_fifth_relations = fourth_fifth_relations[
    fourth_fifth_relations.RxNorm_concept_id_4!=fourth_fifth_relations.RxNorm_concept_id_5
]
print(fourth_fifth_relations.shape)
fourth_fifth_relations = (fourth_fifth_relations.
                          reindex(np.sort(fourth_fifth_relations.columns),
                                  axis=1)
                         )
print(fourth_fifth_relations.head())

(353036, 9)
   RxNorm_concept_class_id_4 RxNorm_concept_class_id_5 RxNorm_concept_code_4  \
2         Clinical Drug Comp                Ingredient                705903   
3         Clinical Drug Form                Ingredient               1658124   
4                 Ingredient        Clinical Drug Comp                 26744   
9                 Ingredient        Clinical Drug Comp                 26744   
10             Clinical Drug        Clinical Drug Comp               1658166   

   RxNorm_concept_code_5  RxNorm_concept_id_4  RxNorm_concept_id_5  \
2                  26744             19126484               501343   
3                  26744             46275211               501343   
4                 353465               501343               501345   
9                 825152               501343               501421   
10                825152             46275251               501421   

                               RxNorm_concept_name_4  \
2             hepatitis B immu

In [165]:
(fourth_fifth_relations.loc[:,['RxNorm_concept_class_id_4','RxNorm_concept_class_id_5']].
groupby(['RxNorm_concept_class_id_4','RxNorm_concept_class_id_5']).
 count()
)

Empty DataFrame
Columns: []
Index: [(Branded Drug, Branded Drug Comp), (Branded Drug, Branded Drug Form), (Branded Drug, Branded Pack), (Branded Drug, Clinical Drug), (Branded Drug, Quant Branded Drug), (Branded Drug Comp, Branded Drug), (Branded Drug Comp, Clinical Drug Comp), (Branded Drug Form, Branded Drug), (Branded Drug Form, Clinical Drug Form), (Branded Pack, Branded Drug), (Branded Pack, Clinical Drug), (Branded Pack, Clinical Pack), (Branded Pack, Quant Branded Drug), (Branded Pack, Quant Clinical Drug), (Clinical Drug, Branded Drug), (Clinical Drug, Branded Pack), (Clinical Drug, Clinical Drug Comp), (Clinical Drug, Clinical Drug Form), (Clinical Drug, Clinical Pack), (Clinical Drug, Quant Clinical Drug), (Clinical Drug Comp, Branded Drug Comp), (Clinical Drug Comp, Clinical Drug), (Clinical Drug Comp, Ingredient), (Clinical Drug Form, Branded Drug Form), (Clinical Drug Form, Clinical Drug), (Clinical Drug Form, Ingredient), (Clinical Pack, Branded Pack), (Clinical Pack, Clinical Drug), (Clinical Pack, Quant Clinical Drug), (Ingredient, Clinical Drug Comp), (Ingredient, Clinical Drug Form), (Quant Branded Drug, Branded Drug), (Quant Branded Drug, Branded Pack), (Quant Branded Drug, Quant Clinical Drug), (Quant Clinical Drug, Branded Pack), (Quant Clinical Drug, Clinical Drug), (Quant Clinical Drug, Clinical Pack), (Quant Clinical Drug, Quant Branded Drug)]

In [166]:
ids = fourth_fifth_relations.RxNorm_concept_id_4.astype(int).unique()

fifth_sixth_relations = (r.
                          query('concept_id_1 in @ids').
                          set_index('concept_id_1').
                          join(c.
                               set_index('concept_id')).
                          rename(
                              columns={
                                  'concept_id_1' : 'RxNorm_concept_id_5',
                                  'concept_code' : 'RxNorm_concept_code_5',
                                  'concept_class_id' : 'RxNorm_concept_class_id_5',
                                  'concept_name' : 'RxNorm_concept_name_5'
                              }
                          ).
                          rename_axis('RxNorm_concept_id_5').
                          reset_index().
                          set_index('concept_id_2').
                          join(c.
                               set_index('concept_id')
                              ).
                          rename(
                              columns={'concept_id_2' : 'RxNorm_concept_id_6',
                                       'concept_code' : 'RxNorm_concept_code_6',
                                       'concept_class_id' : 'RxNorm_concept_class_id_6',
                                       'concept_name' : 'RxNorm_concept_name_6',
                                       'relationship_id' :'relationship_id_56'
                                      }
                          ).
                          rename_axis('RxNorm_concept_id_6').
                          reset_index().
                          dropna().
                          drop_duplicates()
                         )
fifth_sixth_relations = fifth_sixth_relations[
    fifth_sixth_relations.RxNorm_concept_id_5!= fifth_sixth_relations.RxNorm_concept_id_6
]
print(fifth_sixth_relations.shape)
fifth_sixth_relations = (fifth_sixth_relations.
                          reindex(np.sort(fifth_sixth_relations.columns),
                                  axis=1)
                         )
print(fifth_sixth_relations.head())

(353036, 9)
   RxNorm_concept_class_id_5 RxNorm_concept_class_id_6 RxNorm_concept_code_5  \
2         Clinical Drug Comp                Ingredient                705903   
3         Clinical Drug Form                Ingredient               1658124   
4                 Ingredient        Clinical Drug Comp                 26744   
9                 Ingredient        Clinical Drug Comp                 26744   
10             Clinical Drug        Clinical Drug Comp               1658166   

   RxNorm_concept_code_6  RxNorm_concept_id_5  RxNorm_concept_id_6  \
2                  26744             19126484               501343   
3                  26744             46275211               501343   
4                 353465               501343               501345   
9                 825152               501343               501421   
10                825152             46275251               501421   

                               RxNorm_concept_name_5  \
2             hepatitis B immu

In [167]:
(fifth_sixth_relations.loc[:,['RxNorm_concept_class_id_5','RxNorm_concept_class_id_6']].
groupby(['RxNorm_concept_class_id_5','RxNorm_concept_class_id_6']).
 count()
)

Empty DataFrame
Columns: []
Index: [(Branded Drug, Branded Drug Comp), (Branded Drug, Branded Drug Form), (Branded Drug, Branded Pack), (Branded Drug, Clinical Drug), (Branded Drug, Quant Branded Drug), (Branded Drug Comp, Branded Drug), (Branded Drug Comp, Clinical Drug Comp), (Branded Drug Form, Branded Drug), (Branded Drug Form, Clinical Drug Form), (Branded Pack, Branded Drug), (Branded Pack, Clinical Drug), (Branded Pack, Clinical Pack), (Branded Pack, Quant Branded Drug), (Branded Pack, Quant Clinical Drug), (Clinical Drug, Branded Drug), (Clinical Drug, Branded Pack), (Clinical Drug, Clinical Drug Comp), (Clinical Drug, Clinical Drug Form), (Clinical Drug, Clinical Pack), (Clinical Drug, Quant Clinical Drug), (Clinical Drug Comp, Branded Drug Comp), (Clinical Drug Comp, Clinical Drug), (Clinical Drug Comp, Ingredient), (Clinical Drug Form, Branded Drug Form), (Clinical Drug Form, Clinical Drug), (Clinical Drug Form, Ingredient), (Clinical Pack, Branded Pack), (Clinical Pack, Clinical Drug), (Clinical Pack, Quant Clinical Drug), (Ingredient, Clinical Drug Comp), (Ingredient, Clinical Drug Form), (Quant Branded Drug, Branded Drug), (Quant Branded Drug, Branded Pack), (Quant Branded Drug, Quant Clinical Drug), (Quant Clinical Drug, Branded Pack), (Quant Clinical Drug, Clinical Drug), (Quant Clinical Drug, Clinical Pack), (Quant Clinical Drug, Quant Branded Drug)]

In [168]:
rxnorm_to_ings123 = (first_second_relations.
 set_index(['RxNorm_concept_id_2','RxNorm_concept_code_2',
            'RxNorm_concept_name_2','RxNorm_concept_class_id_2']).
 join(second_third_relations.
      set_index(['RxNorm_concept_id_2','RxNorm_concept_code_2',
                 'RxNorm_concept_name_2','RxNorm_concept_class_id_2'])
     ).
 query('RxNorm_concept_class_id_3=="Ingredient" & '+
       '(RxNorm_concept_class_id_1!=RxNorm_concept_class_id_3)').
                  reset_index()
)
print(rxnorm_to_ings123.shape)
print(rxnorm_to_ings123.head())

(21437, 14)
   RxNorm_concept_id_2 RxNorm_concept_code_2 RxNorm_concept_name_2  \
0               700501                328731      Thiothixene 1 MG   
1               700522                328732      Thiothixene 5 MG   
2               700523                328733     Thiothixene 10 MG   
3               702994                316243    Meprobamate 200 MG   
4               702995                316244    Meprobamate 400 MG   

  RxNorm_concept_class_id_2 RxNorm_concept_class_id_1 RxNorm_concept_code_1  \
0        Clinical Drug Comp             Clinical Drug                313362   
1        Clinical Drug Comp             Clinical Drug                313366   
2        Clinical Drug Comp             Clinical Drug                313361   
3        Clinical Drug Comp             Clinical Drug                197928   
4        Clinical Drug Comp             Clinical Drug                197929   

   RxNorm_concept_id_1           RxNorm_concept_name_1 relationship_id_12  \
0              

In [169]:
len(np.intersect1d(
    rxnorm_to_ings123.RxNorm_concept_id_1.dropna().astype(int).unique(),
    all_openFDA_rxnorm_concept_ids
))/len(all_openFDA_rxnorm_concept_ids)

0.47829906870497274

In [170]:
(rxnorm_to_ings123.
loc[:,['RxNorm_concept_name_1','RxNorm_concept_name_3']].
drop_duplicates()
).head()

,RxNorm_concept_name_1,RxNorm_concept_name_3
0,Thiothixene 1 MG Oral Capsule,Thiothixene
1,Thiothixene 5 MG Oral Capsule,Thiothixene
2,Thiothixene 10 MG Oral Capsule,Thiothixene
3,Meprobamate 200 MG Oral Tablet,Meprobamate
4,Meprobamate 400 MG Oral Tablet,Meprobamate


In [171]:
(rxnorm_to_ings123.
loc[:,['RxNorm_concept_class_id_1','RxNorm_concept_class_id_2',
       'RxNorm_concept_class_id_3']].
 drop_duplicates()
)

,RxNorm_concept_class_id_1,RxNorm_concept_class_id_2,RxNorm_concept_class_id_3
0,Clinical Drug,Clinical Drug Comp,Ingredient
245,Clinical Drug,Clinical Drug Form,Ingredient


In [172]:
rxnorm_to_ings123_to_add = (rxnorm_to_ings123.
loc[:,['RxNorm_concept_id_1','RxNorm_concept_code_1',
       'RxNorm_concept_name_1','RxNorm_concept_class_id_1',
       'RxNorm_concept_id_3','RxNorm_concept_code_3',
       'RxNorm_concept_name_3','RxNorm_concept_class_id_3']].
 drop_duplicates().
 rename(
     columns={
         'RxNorm_concept_id_3' : 'RxNorm_concept_id_2',
         'RxNorm_concept_code_3' : 'RxNorm_concept_code_2',
         'RxNorm_concept_name_3' : 'RxNorm_concept_name_2',
         'RxNorm_concept_class_id_3' : 'RxNorm_concept_class_id_2'
     })
                            .drop_duplicates()
)
print(rxnorm_to_ings123_to_add.shape)
rxnorm_to_ings123_to_add.head()

(10706, 8)


,RxNorm_concept_id_1,RxNorm_concept_code_1,RxNorm_concept_name_1,RxNorm_concept_class_id_1,RxNorm_concept_id_2,RxNorm_concept_code_2,RxNorm_concept_name_2,RxNorm_concept_class_id_2
0,700494,313362,Thiothixene 1 MG Oral Capsule,Clinical Drug,700465,10510,Thiothixene,Ingredient
1,700498,313366,Thiothixene 5 MG Oral Capsule,Clinical Drug,700465,10510,Thiothixene,Ingredient
2,700493,313361,Thiothixene 10 MG Oral Capsule,Clinical Drug,700465,10510,Thiothixene,Ingredient
3,702867,197928,Meprobamate 200 MG Oral Tablet,Clinical Drug,702865,6760,Meprobamate,Ingredient
4,702868,197929,Meprobamate 400 MG Oral Tablet,Clinical Drug,702865,6760,Meprobamate,Ingredient


In [173]:
rxnorm_to_ings1234 = (first_second_relations.
 set_index(['RxNorm_concept_id_2','RxNorm_concept_code_2',
            'RxNorm_concept_name_2','RxNorm_concept_class_id_2']).
 join(second_third_relations.
      set_index(['RxNorm_concept_id_2','RxNorm_concept_code_2',
                 'RxNorm_concept_name_2','RxNorm_concept_class_id_2'])
     ).
 query('RxNorm_concept_class_id_3!="Ingredient" & '+
       '(RxNorm_concept_class_id_1!=RxNorm_concept_class_id_3)').
                  reset_index().
                      set_index(
                          ['RxNorm_concept_id_3','RxNorm_concept_code_3',
                           'RxNorm_concept_name_3','RxNorm_concept_class_id_3']
                      ).
                      join(third_fourth_relations.
                          set_index(
                          ['RxNorm_concept_id_3','RxNorm_concept_code_3',
                           'RxNorm_concept_name_3','RxNorm_concept_class_id_3']
                          )
                          ).
 query('RxNorm_concept_class_id_4=="Ingredient"').
                      reset_index()
)
rxnorm_to_ings1234 = rxnorm_to_ings1234.reindex(np.sort(rxnorm_to_ings1234.columns),axis=1)
print(rxnorm_to_ings1234.shape)
rxnorm_to_ings1234.head()

(34724, 19)


,RxNorm_concept_class_id_1,RxNorm_concept_class_id_2,RxNorm_concept_class_id_3,RxNorm_concept_class_id_4,RxNorm_concept_code_1,RxNorm_concept_code_2,RxNorm_concept_code_3,RxNorm_concept_code_4,RxNorm_concept_id_1,RxNorm_concept_id_2,RxNorm_concept_id_3,RxNorm_concept_id_4,RxNorm_concept_name_1,RxNorm_concept_name_2,RxNorm_concept_name_3,RxNorm_concept_name_4,relationship_id_12,relationship_id_23,relationship_id_34
0,Quant Clinical Drug,Clinical Drug,Clinical Drug Comp,Ingredient,1809414,1809417,343818,5666,40220524,40220527,551822,19117912,"50 ML Immunoglobulin G, Human 100 MG/ML Injection","Immunoglobulin G, Human 100 MG/ML Injection","Immunoglobulin G, Human 100 MG/ML",Immunoglobulin G,Quantified form of,Consists of,RxNorm has ing
1,Quant Clinical Drug,Clinical Drug,Clinical Drug Comp,Ingredient,1809421,1809417,343818,5666,40220529,40220527,551822,19117912,"100 ML Immunoglobulin G, Human 100 MG/ML Injec...","Immunoglobulin G, Human 100 MG/ML Injection","Immunoglobulin G, Human 100 MG/ML",Immunoglobulin G,Quantified form of,Consists of,RxNorm has ing
2,Quant Clinical Drug,Clinical Drug,Clinical Drug Comp,Ingredient,672567,844993,672565,679314,703246,19134800,703245,703244,24 HR paliperidone 3 MG Extended Release Oral ...,paliperidone 3 MG Extended Release Oral Tablet,paliperidone 3 MG,paliperidone,Quantified form of,Consists of,RxNorm has ing
3,Quant Clinical Drug,Clinical Drug,Clinical Drug Comp,Ingredient,672569,844994,672568,679314,703248,19134801,703247,703244,24 HR paliperidone 6 MG Extended Release Oral ...,paliperidone 6 MG Extended Release Oral Tablet,paliperidone 6 MG,paliperidone,Quantified form of,Consists of,RxNorm has ing
4,Branded Drug,Clinical Drug,Clinical Drug Comp,Ingredient,104838,313447,328687,10734,19004820,703544,703545,703470,Tranylcypromine 10 MG Oral Tablet [Parnate],Tranylcypromine 10 MG Oral Tablet,Tranylcypromine 10 MG,Tranylcypromine,Tradename of,Consists of,RxNorm has ing


In [174]:
(rxnorm_to_ings1234.
loc[:,['RxNorm_concept_name_1','RxNorm_concept_name_4']].
drop_duplicates()
).head()
len(np.intersect1d(rxnorm_to_ings1234.RxNorm_concept_id_1.dropna().astype(int).unique(),
                  all_openFDA_rxnorm_concept_ids
                  ))/len(all_openFDA_rxnorm_concept_ids)

0.43144145727171557

In [175]:
(rxnorm_to_ings1234.
loc[:,['RxNorm_concept_class_id_1','RxNorm_concept_class_id_2',
       'RxNorm_concept_class_id_3','RxNorm_concept_class_id_4']].
 drop_duplicates()
)

,RxNorm_concept_class_id_1,RxNorm_concept_class_id_2,RxNorm_concept_class_id_3,RxNorm_concept_class_id_4
0,Quant Clinical Drug,Clinical Drug,Clinical Drug Comp,Ingredient
4,Branded Drug,Clinical Drug,Clinical Drug Comp,Ingredient
5,Branded Drug,Branded Drug Comp,Clinical Drug Comp,Ingredient
14,Clinical Pack,Clinical Drug,Clinical Drug Comp,Ingredient
446,Branded Drug,Branded Drug Form,Clinical Drug Form,Ingredient
448,Branded Drug,Clinical Drug,Clinical Drug Form,Ingredient
471,Quant Clinical Drug,Clinical Drug,Clinical Drug Form,Ingredient
1155,Branded Pack,Clinical Drug,Clinical Drug Comp,Ingredient
2087,Clinical Pack,Clinical Drug,Clinical Drug Form,Ingredient
2089,Branded Pack,Clinical Drug,Clinical Drug Form,Ingredient


In [176]:
rxnorm_to_ings1234_to_add = (rxnorm_to_ings1234.
loc[:,['RxNorm_concept_id_1','RxNorm_concept_code_1',
       'RxNorm_concept_name_1','RxNorm_concept_class_id_1',
       'RxNorm_concept_id_4','RxNorm_concept_code_4',
       'RxNorm_concept_name_4','RxNorm_concept_class_id_4']].
 drop_duplicates().
 rename(
     columns={
         'RxNorm_concept_id_4' : 'RxNorm_concept_id_2',
         'RxNorm_concept_code_4' : 'RxNorm_concept_code_2',
         'RxNorm_concept_name_4' : 'RxNorm_concept_name_2',
         'RxNorm_concept_class_id_4' : 'RxNorm_concept_class_id_2'
     })
                            .drop_duplicates()
)
print(rxnorm_to_ings1234_to_add.shape)
rxnorm_to_ings1234_to_add.head()

(9994, 8)


,RxNorm_concept_id_1,RxNorm_concept_code_1,RxNorm_concept_name_1,RxNorm_concept_class_id_1,RxNorm_concept_id_2,RxNorm_concept_code_2,RxNorm_concept_name_2,RxNorm_concept_class_id_2
0,40220524,1809414,"50 ML Immunoglobulin G, Human 100 MG/ML Injection",Quant Clinical Drug,19117912,5666,Immunoglobulin G,Ingredient
1,40220529,1809421,"100 ML Immunoglobulin G, Human 100 MG/ML Injec...",Quant Clinical Drug,19117912,5666,Immunoglobulin G,Ingredient
2,703246,672567,24 HR paliperidone 3 MG Extended Release Oral ...,Quant Clinical Drug,703244,679314,paliperidone,Ingredient
3,703248,672569,24 HR paliperidone 6 MG Extended Release Oral ...,Quant Clinical Drug,703244,679314,paliperidone,Ingredient
4,19004820,104838,Tranylcypromine 10 MG Oral Tablet [Parnate],Branded Drug,703470,10734,Tranylcypromine,Ingredient


In [177]:
len(
    np.intersect1d(
        np.union1d(
            rxnorm_to_ings123.RxNorm_concept_id_1.dropna().astype(int).unique(),
            rxnorm_to_ings1234.RxNorm_concept_id_1.dropna().astype(int).unique()
        ),
        all_openFDA_rxnorm_concept_ids
    )
                  )/len(all_openFDA_rxnorm_concept_ids)

0.9097405259766883

In [178]:
rxnorm_to_ings12345 = (first_second_relations.
 set_index(['RxNorm_concept_id_2','RxNorm_concept_code_2',
            'RxNorm_concept_name_2','RxNorm_concept_class_id_2']).
 join(second_third_relations.
      set_index(['RxNorm_concept_id_2','RxNorm_concept_code_2',
                 'RxNorm_concept_name_2','RxNorm_concept_class_id_2'])
     ).
 query('RxNorm_concept_class_id_3!="Ingredient" & '+
       '(RxNorm_concept_class_id_1!=RxNorm_concept_class_id_3)').
                  reset_index().
                      set_index(
                          ['RxNorm_concept_id_3','RxNorm_concept_code_3',
                           'RxNorm_concept_name_3','RxNorm_concept_class_id_3']
                      ).
                      join(third_fourth_relations.
                          set_index(
                          ['RxNorm_concept_id_3','RxNorm_concept_code_3',
                           'RxNorm_concept_name_3','RxNorm_concept_class_id_3']
                          )
                          ).
 query('RxNorm_concept_class_id_4!="Ingredient" & '+
       '(RxNorm_concept_class_id_2!=RxNorm_concept_class_id_4)').
                      reset_index().
                      set_index(
                          ['RxNorm_concept_id_4','RxNorm_concept_code_4',
                           'RxNorm_concept_name_4','RxNorm_concept_class_id_4']
                      ).
                      join(fourth_fifth_relations.
                          set_index(
                          ['RxNorm_concept_id_4','RxNorm_concept_code_4',
                           'RxNorm_concept_name_4','RxNorm_concept_class_id_4']
                          )
                          ).
 query('RxNorm_concept_class_id_5=="Ingredient"').
                      reset_index()
)
rxnorm_to_ings12345 = rxnorm_to_ings12345.reindex(np.sort(rxnorm_to_ings12345.columns),axis=1)
print(rxnorm_to_ings12345.shape)
rxnorm_to_ings12345.head()

(41391, 24)


,RxNorm_concept_class_id_1,RxNorm_concept_class_id_2,RxNorm_concept_class_id_3,RxNorm_concept_class_id_4,RxNorm_concept_class_id_5,RxNorm_concept_code_1,RxNorm_concept_code_2,RxNorm_concept_code_3,RxNorm_concept_code_4,RxNorm_concept_code_5,...,RxNorm_concept_id_5,RxNorm_concept_name_1,RxNorm_concept_name_2,RxNorm_concept_name_3,RxNorm_concept_name_4,RxNorm_concept_name_5,relationship_id_12,relationship_id_23,relationship_id_34,relationship_id_45
0,Quant Branded Drug,Quant Clinical Drug,Clinical Drug,Clinical Drug Comp,Ingredient,1809416,1809414,1809417,343818,5666,...,19117912,"50 ML Immunoglobulin G, Human 100 MG/ML Inject...","50 ML Immunoglobulin G, Human 100 MG/ML Injection","Immunoglobulin G, Human 100 MG/ML Injection","Immunoglobulin G, Human 100 MG/ML",Immunoglobulin G,Tradename of,Quantified form of,Consists of,RxNorm has ing
1,Quant Branded Drug,Branded Drug,Clinical Drug,Clinical Drug Comp,Ingredient,1809416,1809418,1809417,343818,5666,...,19117912,"50 ML Immunoglobulin G, Human 100 MG/ML Inject...","Immunoglobulin G, Human 100 MG/ML Injection [B...","Immunoglobulin G, Human 100 MG/ML Injection","Immunoglobulin G, Human 100 MG/ML",Immunoglobulin G,Quantified form of,Tradename of,Consists of,RxNorm has ing
2,Quant Branded Drug,Branded Drug,Clinical Drug,Clinical Drug Comp,Ingredient,1809422,1809418,1809417,343818,5666,...,19117912,"100 ML Immunoglobulin G, Human 100 MG/ML Injec...","Immunoglobulin G, Human 100 MG/ML Injection [B...","Immunoglobulin G, Human 100 MG/ML Injection","Immunoglobulin G, Human 100 MG/ML",Immunoglobulin G,Quantified form of,Tradename of,Consists of,RxNorm has ing
3,Quant Branded Drug,Quant Clinical Drug,Clinical Drug,Clinical Drug Comp,Ingredient,1809422,1809421,1809417,343818,5666,...,19117912,"100 ML Immunoglobulin G, Human 100 MG/ML Injec...","100 ML Immunoglobulin G, Human 100 MG/ML Injec...","Immunoglobulin G, Human 100 MG/ML Injection","Immunoglobulin G, Human 100 MG/ML",Immunoglobulin G,Tradename of,Quantified form of,Consists of,RxNorm has ing
4,Quant Branded Drug,Branded Drug,Branded Drug Comp,Clinical Drug Comp,Ingredient,1809416,1809418,1364425,343818,5666,...,19117912,"50 ML Immunoglobulin G, Human 100 MG/ML Inject...","Immunoglobulin G, Human 100 MG/ML Injection [B...","Immunoglobulin G, Human 100 MG/ML [Bivigam]","Immunoglobulin G, Human 100 MG/ML",Immunoglobulin G,Quantified form of,Consists of,Tradename of,RxNorm has ing


In [179]:
(rxnorm_to_ings12345.
loc[:,['RxNorm_concept_name_1','RxNorm_concept_name_5']].
drop_duplicates()
).head()
len(np.intersect1d(rxnorm_to_ings12345.RxNorm_concept_id_1.dropna().astype(int).unique(),
                  all_openFDA_rxnorm_concept_ids
                  ))/len(all_openFDA_rxnorm_concept_ids)

0.4132841328413284

In [180]:
(rxnorm_to_ings12345.
loc[:,['RxNorm_concept_class_id_1','RxNorm_concept_class_id_2',
       'RxNorm_concept_class_id_3','RxNorm_concept_class_id_4',
       'RxNorm_concept_class_id_5']].
 drop_duplicates()
)

,RxNorm_concept_class_id_1,RxNorm_concept_class_id_2,RxNorm_concept_class_id_3,RxNorm_concept_class_id_4,RxNorm_concept_class_id_5
0,Quant Branded Drug,Quant Clinical Drug,Clinical Drug,Clinical Drug Comp,Ingredient
1,Quant Branded Drug,Branded Drug,Clinical Drug,Clinical Drug Comp,Ingredient
4,Quant Branded Drug,Branded Drug,Branded Drug Comp,Clinical Drug Comp,Ingredient
6,Clinical Drug,Branded Drug,Branded Drug Comp,Clinical Drug Comp,Ingredient
35,Branded Pack,Branded Drug,Clinical Drug,Clinical Drug Comp,Ingredient
37,Branded Pack,Clinical Pack,Clinical Drug,Clinical Drug Comp,Ingredient
48,Branded Pack,Branded Drug,Branded Drug Comp,Clinical Drug Comp,Ingredient
55,Clinical Pack,Quant Clinical Drug,Clinical Drug,Clinical Drug Comp,Ingredient
692,Clinical Drug,Branded Drug,Branded Drug Form,Clinical Drug Form,Ingredient
709,Quant Branded Drug,Branded Drug,Branded Drug Form,Clinical Drug Form,Ingredient


In [181]:
rxnorm_to_ings12345_to_add = (rxnorm_to_ings12345.
loc[:,['RxNorm_concept_id_1','RxNorm_concept_code_1',
       'RxNorm_concept_name_1','RxNorm_concept_class_id_1',
       'RxNorm_concept_id_5','RxNorm_concept_code_5',
       'RxNorm_concept_name_5','RxNorm_concept_class_id_5']].
 drop_duplicates().
 rename(
     columns={
         'RxNorm_concept_id_5' : 'RxNorm_concept_id_2',
         'RxNorm_concept_code_5' : 'RxNorm_concept_code_2',
         'RxNorm_concept_name_5' : 'RxNorm_concept_name_2',
         'RxNorm_concept_class_id_5' : 'RxNorm_concept_class_id_2'
     })
                            .drop_duplicates()
)
print(rxnorm_to_ings12345_to_add.shape)
rxnorm_to_ings12345_to_add.head()

(9588, 8)


,RxNorm_concept_id_1,RxNorm_concept_code_1,RxNorm_concept_name_1,RxNorm_concept_class_id_1,RxNorm_concept_id_2,RxNorm_concept_code_2,RxNorm_concept_name_2,RxNorm_concept_class_id_2
0,40220526,1809416,"50 ML Immunoglobulin G, Human 100 MG/ML Inject...",Quant Branded Drug,19117912,5666,Immunoglobulin G,Ingredient
2,40220530,1809422,"100 ML Immunoglobulin G, Human 100 MG/ML Injec...",Quant Branded Drug,19117912,5666,Immunoglobulin G,Ingredient
6,700494,313362,Thiothixene 1 MG Oral Capsule,Clinical Drug,700465,10510,Thiothixene,Ingredient
7,700498,313366,Thiothixene 5 MG Oral Capsule,Clinical Drug,700465,10510,Thiothixene,Ingredient
8,700493,313361,Thiothixene 10 MG Oral Capsule,Clinical Drug,700465,10510,Thiothixene,Ingredient


In [182]:
len(
    np.intersect1d(
        np.union1d(
            np.union1d(
                rxnorm_to_ings123.RxNorm_concept_id_1.dropna().astype(int).unique(),
                rxnorm_to_ings1234.RxNorm_concept_id_1.dropna().astype(int).unique()
            ),
            rxnorm_to_ings12345.RxNorm_concept_id_1.dropna().astype(int).unique()
        ),
        all_openFDA_rxnorm_concept_ids
    )
                  )/len(all_openFDA_rxnorm_concept_ids)

0.9977742634569203

In [183]:
np.setdiff1d(
        all_openFDA_rxnorm_concept_ids,
    np.union1d(
            np.union1d(
                rxnorm_to_ings123.RxNorm_concept_id_1.dropna().astype(int).unique(),
                rxnorm_to_ings1234.RxNorm_concept_id_1.dropna().astype(int).unique()
            ),
            rxnorm_to_ings12345.RxNorm_concept_id_1.dropna().astype(int).unique()
        )
    )

array([ 1592626,  1592627,  1592628,  1592637,  1592638,  1594443,
        1594444,  1594468,  1594469,  1594470,  1594471,  1594472,
        1594473,  1594484,  1594485,  1594486,  1594487,  1718880,
        1718886, 19130130, 19133087, 35201871, 35201893, 36249660,
       40172829, 40172833, 40172920, 40220770, 40223041, 40237386,
       43011780, 43560401, 45777047, 45777078, 45777080, 46234547,
       46234550, 46234551])

In [184]:
rxnorm_to_ings123456 = (first_second_relations.
 set_index(['RxNorm_concept_id_2','RxNorm_concept_code_2',
            'RxNorm_concept_name_2','RxNorm_concept_class_id_2']).
 join(second_third_relations.
      set_index(['RxNorm_concept_id_2','RxNorm_concept_code_2',
                 'RxNorm_concept_name_2','RxNorm_concept_class_id_2'])
     ).
 query('RxNorm_concept_class_id_3!="Ingredient" & '+
       '(RxNorm_concept_class_id_1!=RxNorm_concept_class_id_3)').
                  reset_index().
                      set_index(
                          ['RxNorm_concept_id_3','RxNorm_concept_code_3',
                           'RxNorm_concept_name_3','RxNorm_concept_class_id_3']
                      ).
                      join(third_fourth_relations.
                          set_index(
                          ['RxNorm_concept_id_3','RxNorm_concept_code_3',
                           'RxNorm_concept_name_3','RxNorm_concept_class_id_3']
                          )
                          ).
 query('RxNorm_concept_class_id_4!="Ingredient" & '+
       '(RxNorm_concept_class_id_2!=RxNorm_concept_class_id_4)').
                      reset_index().
                      set_index(
                          ['RxNorm_concept_id_4','RxNorm_concept_code_4',
                           'RxNorm_concept_name_4','RxNorm_concept_class_id_4']
                      ).
                      join(fourth_fifth_relations.
                          set_index(
                          ['RxNorm_concept_id_4','RxNorm_concept_code_4',
                           'RxNorm_concept_name_4','RxNorm_concept_class_id_4']
                          )
                          ).
 query('RxNorm_concept_class_id_5!="Ingredient" & '+
       '(RxNorm_concept_class_id_3!=RxNorm_concept_class_id_5)').
                      reset_index().
                      set_index(
                          ['RxNorm_concept_id_5','RxNorm_concept_code_5',
                           'RxNorm_concept_name_5','RxNorm_concept_class_id_5']
                      ).
                      join(fifth_sixth_relations.
                          set_index(
                          ['RxNorm_concept_id_5','RxNorm_concept_code_5',
                           'RxNorm_concept_name_5','RxNorm_concept_class_id_5']
                          )
                          ).
 query('RxNorm_concept_class_id_6=="Ingredient"').
                      reset_index()
)
rxnorm_to_ings123456 = rxnorm_to_ings123456.reindex(np.sort(rxnorm_to_ings123456.columns),axis=1)
print(rxnorm_to_ings123456.shape)
rxnorm_to_ings123456.head()

(298952, 29)


,RxNorm_concept_class_id_1,RxNorm_concept_class_id_2,RxNorm_concept_class_id_3,RxNorm_concept_class_id_4,RxNorm_concept_class_id_5,RxNorm_concept_class_id_6,RxNorm_concept_code_1,RxNorm_concept_code_2,RxNorm_concept_code_3,RxNorm_concept_code_4,...,RxNorm_concept_name_2,RxNorm_concept_name_3,RxNorm_concept_name_4,RxNorm_concept_name_5,RxNorm_concept_name_6,relationship_id_12,relationship_id_23,relationship_id_34,relationship_id_45,relationship_id_56
0,Quant Clinical Drug,Quant Branded Drug,Branded Drug,Branded Drug Comp,Clinical Drug Comp,Ingredient,1809414,1876671,1876672,1876670,...,"50 ML Immunoglobulin G, Human 100 MG/ML Inject...","Immunoglobulin G, Human 100 MG/ML Injection [G...","Immunoglobulin G, Human 100 MG/ML [Gammaplex]","Immunoglobulin G, Human 100 MG/ML",Immunoglobulin G,Has tradename,Quantified form of,Consists of,Tradename of,RxNorm has ing
1,Quant Clinical Drug,Quant Branded Drug,Branded Drug,Branded Drug Comp,Clinical Drug Comp,Ingredient,1809421,1876673,1876672,1876670,...,"100 ML Immunoglobulin G, Human 100 MG/ML Injec...","Immunoglobulin G, Human 100 MG/ML Injection [G...","Immunoglobulin G, Human 100 MG/ML [Gammaplex]","Immunoglobulin G, Human 100 MG/ML",Immunoglobulin G,Has tradename,Quantified form of,Consists of,Tradename of,RxNorm has ing
2,Quant Clinical Drug,Clinical Drug,Branded Drug,Branded Drug Comp,Clinical Drug Comp,Ingredient,1809414,1809417,1876672,1876670,...,"Immunoglobulin G, Human 100 MG/ML Injection","Immunoglobulin G, Human 100 MG/ML Injection [G...","Immunoglobulin G, Human 100 MG/ML [Gammaplex]","Immunoglobulin G, Human 100 MG/ML",Immunoglobulin G,Quantified form of,Has tradename,Consists of,Tradename of,RxNorm has ing
3,Quant Clinical Drug,Clinical Drug,Branded Drug,Branded Drug Comp,Clinical Drug Comp,Ingredient,1809421,1809417,1876672,1876670,...,"Immunoglobulin G, Human 100 MG/ML Injection","Immunoglobulin G, Human 100 MG/ML Injection [G...","Immunoglobulin G, Human 100 MG/ML [Gammaplex]","Immunoglobulin G, Human 100 MG/ML",Immunoglobulin G,Quantified form of,Has tradename,Consists of,Tradename of,RxNorm has ing
4,Quant Clinical Drug,Clinical Drug,Branded Drug,Branded Drug Comp,Clinical Drug Comp,Ingredient,1809414,1809417,1809518,758997,...,"Immunoglobulin G, Human 100 MG/ML Injection","Immunoglobulin G, Human 100 MG/ML Injection [P...","Immunoglobulin G, Human 100 MG/ML [Privigen]","Immunoglobulin G, Human 100 MG/ML",Immunoglobulin G,Quantified form of,Has tradename,Consists of,Tradename of,RxNorm has ing


In [185]:
(rxnorm_to_ings123456.
loc[:,['RxNorm_concept_name_1','RxNorm_concept_name_6']].
drop_duplicates()
).head()
len(np.intersect1d(rxnorm_to_ings123456.RxNorm_concept_id_1.dropna().astype(int).unique(),
                  all_openFDA_rxnorm_concept_ids
                  ))/len(all_openFDA_rxnorm_concept_ids)

0.4054354829262578

In [186]:
(rxnorm_to_ings123456.
loc[:,['RxNorm_concept_class_id_1','RxNorm_concept_class_id_2',
       'RxNorm_concept_class_id_3','RxNorm_concept_class_id_4',
       'RxNorm_concept_class_id_5','RxNorm_concept_class_id_6']].
 drop_duplicates()
)

,RxNorm_concept_class_id_1,RxNorm_concept_class_id_2,RxNorm_concept_class_id_3,RxNorm_concept_class_id_4,RxNorm_concept_class_id_5,RxNorm_concept_class_id_6
0,Quant Clinical Drug,Quant Branded Drug,Branded Drug,Branded Drug Comp,Clinical Drug Comp,Ingredient
2,Quant Clinical Drug,Clinical Drug,Branded Drug,Branded Drug Comp,Clinical Drug Comp,Ingredient
16,Quant Clinical Drug,Quant Branded Drug,Branded Drug,Clinical Drug,Clinical Drug Comp,Ingredient
54,Branded Drug,Branded Drug Form,Clinical Drug Form,Clinical Drug,Clinical Drug Comp,Ingredient
65,Clinical Pack,Branded Pack,Branded Drug,Clinical Drug,Clinical Drug Comp,Ingredient
67,Branded Drug,Branded Pack,Clinical Pack,Clinical Drug,Clinical Drug Comp,Ingredient
96,Clinical Pack,Clinical Drug,Branded Drug,Branded Drug Comp,Clinical Drug Comp,Ingredient
98,Clinical Pack,Branded Pack,Branded Drug,Branded Drug Comp,Clinical Drug Comp,Ingredient
106,Branded Pack,Quant Branded Drug,Branded Drug,Branded Drug Comp,Clinical Drug Comp,Ingredient
108,Branded Pack,Quant Branded Drug,Quant Clinical Drug,Clinical Drug,Clinical Drug Comp,Ingredient


In [187]:
rxnorm_to_ings123456_to_add = (rxnorm_to_ings123456.
loc[:,['RxNorm_concept_id_1','RxNorm_concept_code_1',
       'RxNorm_concept_name_1','RxNorm_concept_class_id_1',
       'RxNorm_concept_id_6','RxNorm_concept_code_6',
       'RxNorm_concept_name_6','RxNorm_concept_class_id_6']].
 drop_duplicates().
 rename(
     columns={
         'RxNorm_concept_id_6' : 'RxNorm_concept_id_2',
         'RxNorm_concept_code_6' : 'RxNorm_concept_code_2',
         'RxNorm_concept_name_6' : 'RxNorm_concept_name_2',
         'RxNorm_concept_class_id_6' : 'RxNorm_concept_class_id_2'
     }).
                            drop_duplicates()
)
print(rxnorm_to_ings123456_to_add.shape)
rxnorm_to_ings123456_to_add.head()

(28995, 8)


,RxNorm_concept_id_1,RxNorm_concept_code_1,RxNorm_concept_name_1,RxNorm_concept_class_id_1,RxNorm_concept_id_2,RxNorm_concept_code_2,RxNorm_concept_name_2,RxNorm_concept_class_id_2
0,40220524,1809414,"50 ML Immunoglobulin G, Human 100 MG/ML Injection",Quant Clinical Drug,19117912.0,5666,Immunoglobulin G,Ingredient
1,40220529,1809421,"100 ML Immunoglobulin G, Human 100 MG/ML Injec...",Quant Clinical Drug,19117912.0,5666,Immunoglobulin G,Ingredient
48,703246,672567,24 HR paliperidone 3 MG Extended Release Oral ...,Quant Clinical Drug,703244.0,679314,paliperidone,Ingredient
51,703248,672569,24 HR paliperidone 6 MG Extended Release Oral ...,Quant Clinical Drug,703244.0,679314,paliperidone,Ingredient
54,19004820,104838,Tranylcypromine 10 MG Oral Tablet [Parnate],Branded Drug,703470.0,10734,Tranylcypromine,Ingredient


In [188]:
len(
    np.intersect1d(
        np.union1d(
            np.union1d(
                np.union1d(
                    rxnorm_to_ings123.RxNorm_concept_id_1.dropna().astype(int).unique(),
                    rxnorm_to_ings1234.RxNorm_concept_id_1.dropna().astype(int).unique()
                ),
                rxnorm_to_ings12345.RxNorm_concept_id_1.dropna().astype(int).unique()
            ),
            rxnorm_to_ings123456.RxNorm_concept_id_1.dropna().astype(int).unique()
        ),
        all_openFDA_rxnorm_concept_ids
    )
                  )/len(all_openFDA_rxnorm_concept_ids)

0.9992971358285011

In [189]:
np.setdiff1d(
        all_openFDA_rxnorm_concept_ids,
        np.union1d(
            np.union1d(
                np.union1d(
                    rxnorm_to_ings123.RxNorm_concept_id_1.dropna().astype(int).unique(),
                    rxnorm_to_ings1234.RxNorm_concept_id_1.dropna().astype(int).unique()
                ),
                rxnorm_to_ings12345.RxNorm_concept_id_1.dropna().astype(int).unique()
            ),
            rxnorm_to_ings123456.RxNorm_concept_id_1.dropna().astype(int).unique()
        )
)

array([1594443, 1594444, 1594468, 1594469, 1594470, 1594471, 1594472,
       1594473, 1594484, 1594485, 1594486, 1594487])

In [190]:
rxnorm_to_ings_all = pd.concat(
    [
        rxnorm_to_ings123_to_add,
        rxnorm_to_ings1234_to_add,
        rxnorm_to_ings12345_to_add,
        rxnorm_to_ings123456_to_add
    ]
).dropna().drop_duplicates()
rxnorm_to_ings_all.RxNorm_concept_id_2 = rxnorm_to_ings_all.RxNorm_concept_id_2.astype(int)
print(rxnorm_to_ings_all.shape)
rxnorm_to_ings_all.head()

(42329, 8)


,RxNorm_concept_id_1,RxNorm_concept_code_1,RxNorm_concept_name_1,RxNorm_concept_class_id_1,RxNorm_concept_id_2,RxNorm_concept_code_2,RxNorm_concept_name_2,RxNorm_concept_class_id_2
0,700494,313362,Thiothixene 1 MG Oral Capsule,Clinical Drug,700465,10510,Thiothixene,Ingredient
1,700498,313366,Thiothixene 5 MG Oral Capsule,Clinical Drug,700465,10510,Thiothixene,Ingredient
2,700493,313361,Thiothixene 10 MG Oral Capsule,Clinical Drug,700465,10510,Thiothixene,Ingredient
3,702867,197928,Meprobamate 200 MG Oral Tablet,Clinical Drug,702865,6760,Meprobamate,Ingredient
4,702868,197929,Meprobamate 400 MG Oral Tablet,Clinical Drug,702865,6760,Meprobamate,Ingredient


In [191]:
len(
    np.intersect1d(
        rxnorm_to_ings_all.RxNorm_concept_id_1,
        all_openFDA_rxnorm_concept_ids
    )
)/len(all_openFDA_rxnorm_concept_ids)

0.9992971358285011

In [202]:
standard_drug = (pd.
                 read_csv(er_dir+'standard_drugs.csv.gz',
                          compression='gzip',
                          dtype={
                              'safetyreportid' : 'str'
                          })
                )
standard_drug.RxNorm_concept_id = standard_drug.RxNorm_concept_id.astype(int)
all_reports = standard_drug.safetyreportid.astype(str).unique()
print(standard_drug.shape)
standard_drug.head()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(10000000, 5)


,RxNorm_concept_class_id,RxNorm_concept_code,RxNorm_concept_id,RxNorm_concept_name,safetyreportid
0,Clinical Drug,91349,1776544,Hydrogen Peroxide 30 MG/ML Topical Solution,10024478
1,Clinical Drug,91349,1776544,Hydrogen Peroxide 30 MG/ML Topical Solution,10037760
2,Clinical Drug,91349,1776544,Hydrogen Peroxide 30 MG/ML Topical Solution,10162316
3,Clinical Drug,91349,1776544,Hydrogen Peroxide 30 MG/ML Topical Solution,10235601
4,Clinical Drug,91349,1776544,Hydrogen Peroxide 30 MG/ML Topical Solution,10284414


In [203]:
standard_drug_ingredients = ((standard_drug.
  loc[:,['RxNorm_concept_id','safetyreportid']].
  drop_duplicates().
set_index(
    [
        'RxNorm_concept_id'
    ]
)
).join(rxnorm_to_ings_all.
       loc[:,['RxNorm_concept_id_1','RxNorm_concept_id_2',
             'RxNorm_concept_code_2','RxNorm_concept_name_2',
             'RxNorm_concept_class_id_2']].
       drop_duplicates().
set_index(
    [
        'RxNorm_concept_id_1'
    ]
)
).drop_duplicates().
 rename(
     columns={
         'RxNorm_concept_id_2':'RxNorm_concept_id',
         'RxNorm_concept_code_2':'RxNorm_concept_code',
         'RxNorm_concept_name_2':'RxNorm_concept_name',
         'RxNorm_concept_class_id_2':'RxNorm_concept_class_id'
     }).
                             reset_index(drop=True).
                             dropna().
                             drop_duplicates()
       )
standard_drug_ingredients = (standard_drug_ingredients.
                             reindex(np.sort(standard_drug_ingredients.columns),axis=1)
                            )
print(standard_drug_ingredients.shape)
standard_drug_ingredients.head()

(9689531, 5)


,RxNorm_concept_class_id,RxNorm_concept_code,RxNorm_concept_id,RxNorm_concept_name,safetyreportid
0,Ingredient,722,713109,Amoxapine,10047430
1,Ingredient,722,713109,Amoxapine,10057255
2,Ingredient,722,713109,Amoxapine,10121390
3,Ingredient,722,713109,Amoxapine,10151205
4,Ingredient,722,713109,Amoxapine,10182055


In [204]:
print(len(
    np.intersect1d(
        standard_drug_ingredients.safetyreportid.astype(str).unique(),
        all_reports
    )
)/len(all_reports))

1.0


In [ ]:
(standard_drug_ingredients.
 to_csv(er_dir+'standard_drugs_rxnorm_ingredients.csv.gz',compression='gzip',index=False))

### standard_reactions_meddra_relationships

In [46]:
standard_reactions = (pd.
                      read_csv(er_dir+'standard_reactions.csv.gz',
                               compression="gzip",
                               dtype={
                                   'safetyreportid' : 'str'
                               }
                              )
                     )
all_reports = (standard_reactions.safetyreportid.unique())
print(standard_reactions.shape)
print(standard_reactions.head())

(1000000, 6)
              MedDRA_concept_name safetyreportid    reaction_outcome  \
0  11-Beta-Hydroxylase Deficiency      7999037-5                 NaN   
1  17 Ketosteroids Urine Abnormal       13978283  Recovered/resolved   
2  17 Ketosteroids Urine Abnormal       13983944  Recovered/resolved   
3  17 Ketosteroids Urine Abnormal       13988153  Recovered/resolved   
4  17 Ketosteroids Urine Abnormal       13993404  Recovered/resolved   

   MedDRA_concept_id  MedDRA_concept_code MedDRA_concept_class_id  
0           35305836             10000002                      PT  
1           36312356             10061608                      PT  
2           36312356             10061608                      PT  
3           36312356             10061608                      PT  
4           36312356             10061608                      PT  


In [47]:
reactions = standard_reactions.MedDRA_concept_id.astype(int).unique()
print(len(reactions))
meddra_concept_ids = concept.query('vocabulary_id=="MedDRA"').concept_id.astype(int).unique()
len(meddra_concept_ids)

intersect = np.intersect1d(reactions,meddra_concept_ids)
print(len(intersect))
print(len(intersect)/len(reactions))

542
542
1.0


In [48]:
meddra_concept = concept.query('vocabulary_id=="MedDRA"')
meddra_concept.concept_id = meddra_concept.concept_id.astype(int)
all_meddra_concept_ids = meddra_concept.concept_id.unique()

r = (concept_relationship.
     copy().
     loc[:,['concept_id_1','concept_id_2','relationship_id']].
     drop_duplicates()
    )
r.concept_id_1 = r.concept_id_1.astype(int)
r.concept_id_2 = r.concept_id_2.astype(int)


In [49]:
(r.
query('concept_id_1 in @all_meddra_concept_ids & '+
     'concept_id_2 in @all_meddra_concept_ids').
relationship_id.value_counts()
)

Subsumes               88624
Is a                   88624
Concept replaced by    21021
Concept replaces       21021
Name: relationship_id, dtype: int64

In [50]:
c = meddra_concept.copy()

all_meddra_relationships = (r.
 query('concept_id_1 in @meddra_concept_ids & '+\
       'concept_id_2 in @meddra_concept_ids').
 set_index('concept_id_1').
 join(
     c. # standard concepts for 1
     query('vocabulary_id=="MedDRA"').
     loc[:,['concept_id','concept_code','concept_name','concept_class_id']].
     drop_duplicates().
     set_index('concept_id')
    ).
 rename_axis('MedDRA_concept_id_1').
 reset_index().
 rename(
     columns={
         'concept_code' : 'MedDRA_concept_code_1',
         'concept_class_id' : 'MedDRA_concept_class_id_1',
         'concept_name' : 'MedDRA_concept_name_1',
         'concept_id_2' : 'MedDRA_concept_id_2',
         'relationship_id' : 'relationship_id_12'
     }
 ).
set_index('MedDRA_concept_id_2').
 join(
     c. # standard concepts for 2
     query('vocabulary_id=="MedDRA"').
     loc[:,['concept_id','concept_code','concept_name','concept_class_id']].
     drop_duplicates().
     set_index('concept_id')
    ).
 rename_axis('MedDRA_concept_id_2').
 reset_index().
 rename(
     columns={
         'concept_code' : 'MedDRA_concept_code_2',
         'concept_class_id' : 'MedDRA_concept_class_id_2',
         'concept_name' : 'MedDRA_concept_name_2'
     }
 ))
all_meddra_relationships = (all_meddra_relationships.
                            reindex(np.sort(all_meddra_relationships.columns),axis=1)
                           )
print(all_meddra_relationships.shape)
print(all_meddra_relationships.head())

(219290, 9)
  MedDRA_concept_class_id_1 MedDRA_concept_class_id_2 MedDRA_concept_code_1  \
0                      HLGT                       SOC              10077537   
1                      HLGT                       SOC              10069782   
2                       SOC                      HLGT              10077536   
3                       HLT                      HLGT              10077538   
4                       HLT                      HLGT              10077539   

  MedDRA_concept_code_2  MedDRA_concept_id_1  MedDRA_concept_id_2  \
0              10077536               788071               788070   
1              10077536             42888891               788070   
2              10077537               788070               788071   
3              10077537               788076               788071   
4              10077537               788077               788071   

                               MedDRA_concept_name_1  \
0  Product quality, supply, distribution, 

In [51]:
print(all_meddra_relationships.MedDRA_concept_class_id_1.value_counts())
print(all_meddra_relationships.MedDRA_concept_class_id_2.value_counts())

PT      106350
LLT      76427
HLT      34026
HLGT      2132
SOC        355
Name: MedDRA_concept_class_id_1, dtype: int64
PT      106350
LLT      76427
HLT      34026
HLGT      2132
SOC        355
Name: MedDRA_concept_class_id_2, dtype: int64


In [52]:
all_meddra_relationships.MedDRA_concept_id_1 = (all_meddra_relationships.
                                                  MedDRA_concept_id_1.
                                                  astype(int)
                                                 )
all_meddra_relationships.MedDRA_concept_code_1 = (all_meddra_relationships.
                                                  MedDRA_concept_code_1.
                                                  astype(int)
                                                 )
all_meddra_relationships.MedDRA_concept_id_2 = (all_meddra_relationships.
                                                  MedDRA_concept_id_2.
                                                  astype(int)
                                                 )
all_meddra_relationships.MedDRA_concept_code_2 = (all_meddra_relationships.
                                                  MedDRA_concept_code_2.
                                                  astype(int)
                                                 )

In [53]:
first_rxs = reactions
first_relations = (all_meddra_relationships.
                   query('MedDRA_concept_id_1 in @first_rxs & '+
                         'MedDRA_concept_class_id_2=="HLT"')
                  ).reset_index(drop=True)
first_relations = (first_relations[
    first_relations.MedDRA_concept_id_1!=first_relations.MedDRA_concept_id_2
])
print(first_relations.shape)
print(first_relations.head())
print(first_relations.MedDRA_concept_class_id_2.value_counts())

(824, 9)
  MedDRA_concept_class_id_1 MedDRA_concept_class_id_2  MedDRA_concept_code_1  \
0                        PT                       HLT               10053745   
1                        PT                       HLT               10056496   
2                        PT                       HLT               10068370   
3                        PT                       HLT               10069495   
4                        PT                       HLT               10074561   

   MedDRA_concept_code_2  MedDRA_concept_id_1  MedDRA_concept_id_2  \
0               10009737             35104129             35102371   
1               10009737             35104130             35102371   
2               10009737             35104131             35102371   
3               10009737             42888942             35102371   
4               10009737             45885651             35102371   

                               MedDRA_concept_name_1  \
0                               A

In [54]:
second_rxs = first_relations.MedDRA_concept_id_2.unique()
second_relations = (all_meddra_relationships.
                    query('MedDRA_concept_id_1 in @second_rxs & '+
                         'MedDRA_concept_class_id_2=="HLGT"').
                    rename(columns={
                        'MedDRA_concept_id_2' : 'MedDRA_concept_id_3',
                        'MedDRA_concept_code_2' : 'MedDRA_concept_code_3',
                        'MedDRA_concept_name_2' : 'MedDRA_concept_name_3',
                        'MedDRA_concept_class_id_2' : 'MedDRA_concept_class_id_3',
                        'MedDRA_concept_id_1' : 'MedDRA_concept_id_2',
                        'MedDRA_concept_code_1' : 'MedDRA_concept_code_2',
                        'MedDRA_concept_name_1' : 'MedDRA_concept_name_2',
                        'MedDRA_concept_class_id_1' : 'MedDRA_concept_class_id_2',
                        'relationship_id_12' : 'relationship_id_23'
                    }
                          )
                  ).reset_index(drop=True)
second_relations = (second_relations[
    second_relations.MedDRA_concept_id_2!=second_relations.MedDRA_concept_id_3
])
print(second_relations.shape)
print(second_relations.head())
print(second_relations.MedDRA_concept_class_id_2.value_counts())
print(second_relations.MedDRA_concept_class_id_3.value_counts())

(385, 9)
  MedDRA_concept_class_id_2 MedDRA_concept_class_id_3  MedDRA_concept_code_2  \
0                       HLT                      HLGT               10009737   
1                       HLT                      HLGT               10053567   
2                       HLT                      HLGT               10037550   
3                       HLT                      HLGT               10018847   
4                       HLT                      HLGT               10018904   

   MedDRA_concept_code_3  MedDRA_concept_id_2  MedDRA_concept_id_3  \
0               10064477             35102371             35102034   
1               10064477             35102372             35102034   
2               10064477             35102373             35102034   
3               10018849             35102374             35102035   
4               10018902             35102380             35102037   

              MedDRA_concept_name_2  \
0   Coagulation factor deficiencies   
1          

In [55]:
third_rxs = second_relations.MedDRA_concept_id_3.unique()
third_relations = (all_meddra_relationships.
                    query('MedDRA_concept_id_1 in @third_rxs & '+
                         'MedDRA_concept_class_id_2=="SOC"').
                    rename(columns={
                        'MedDRA_concept_id_2' : 'MedDRA_concept_id_4',
                        'MedDRA_concept_code_2' : 'MedDRA_concept_code_4',
                        'MedDRA_concept_name_2' : 'MedDRA_concept_name_4',
                        'MedDRA_concept_class_id_2' : 'MedDRA_concept_class_id_4',
                        'MedDRA_concept_id_1' : 'MedDRA_concept_id_3',
                        'MedDRA_concept_code_1' : 'MedDRA_concept_code_3',
                        'MedDRA_concept_name_1' : 'MedDRA_concept_name_3',
                        'MedDRA_concept_class_id_1' : 'MedDRA_concept_class_id_3',
                        'relationship_id_12' : 'relationship_id_34'
                    }
                          )
                  ).reset_index(drop=True)
third_relations = (third_relations[
    third_relations.MedDRA_concept_id_3!=third_relations.MedDRA_concept_id_4
])
print(third_relations.shape)
print(third_relations.head())
print(third_relations.MedDRA_concept_class_id_3.value_counts())
print(third_relations.MedDRA_concept_class_id_4.value_counts())

(203, 9)
  MedDRA_concept_class_id_3 MedDRA_concept_class_id_4  MedDRA_concept_code_3  \
0                      HLGT                       SOC               10069782   
1                      HLGT                       SOC               10064477   
2                      HLGT                       SOC               10018849   
3                      HLGT                       SOC               10018902   
4                      HLGT                       SOC               10018911   

   MedDRA_concept_code_4  MedDRA_concept_id_3  MedDRA_concept_id_4  \
0               10077536             42888891               788070   
1               10005329             35102034             35100000   
2               10005329             35102035             35100000   
3               10005329             35102037             35100000   
4               10005329             35102038             35100000   

                               MedDRA_concept_name_3  \
0                                

In [56]:
first_second_third_relations = \
(first_relations.
 set_index('MedDRA_concept_id_2').
 join(second_relations.
      loc[:,['MedDRA_concept_id_2','MedDRA_concept_id_3',
             'MedDRA_concept_name_3','MedDRA_concept_class_id_3',
             'MedDRA_concept_code_3','relationship_id_23']].
      set_index('MedDRA_concept_id_2')
     ).
 reset_index()
)
first_second_third_relations = \
(first_second_third_relations.
 reindex(np.sort(first_second_third_relations.columns),
         axis=1)
)
first_second_third_relations['MedDRA_concept_id_3'] = \
first_second_third_relations['MedDRA_concept_id_3'].astype(int)
print(first_second_third_relations.shape)
print(first_second_third_relations.head())
print(first_second_third_relations.MedDRA_concept_class_id_1.value_counts())
print(first_second_third_relations.MedDRA_concept_class_id_2.value_counts())
print(first_second_third_relations.MedDRA_concept_class_id_3.value_counts())

(830, 14)
  MedDRA_concept_class_id_1 MedDRA_concept_class_id_2  \
0                        PT                       HLT   
1                        PT                       HLT   
2                        PT                       HLT   
3                        PT                       HLT   
4                        PT                       HLT   

  MedDRA_concept_class_id_3  MedDRA_concept_code_1  MedDRA_concept_code_2  \
0                      HLGT               10053745               10009737   
1                      HLGT               10056496               10009737   
2                      HLGT               10068370               10009737   
3                      HLGT               10069495               10009737   
4                      HLGT               10074561               10009737   

   MedDRA_concept_code_3  MedDRA_concept_id_1  MedDRA_concept_id_2  \
0               10064477             35104129             35102371   
1               10064477             3510413

In [57]:
first_second_third_fourth_relations = \
(first_relations.
 set_index('MedDRA_concept_id_2').
 join(second_relations.
      loc[:,['MedDRA_concept_id_2','MedDRA_concept_id_3',
             'MedDRA_concept_name_3','MedDRA_concept_class_id_3',
             'MedDRA_concept_code_3','relationship_id_23']].
      drop_duplicates().
      set_index('MedDRA_concept_id_2')
     ).
 reset_index().
 set_index('MedDRA_concept_id_3').
 join(third_relations.
      loc[:,['MedDRA_concept_id_3','MedDRA_concept_id_4',
             'MedDRA_concept_name_4','MedDRA_concept_class_id_4',
             'MedDRA_concept_code_4','relationship_id_34']].
      drop_duplicates().
      set_index('MedDRA_concept_id_3')
     ).
 reset_index()
)
first_second_third_fourth_relations = \
(first_second_third_fourth_relations.
 reindex(np.sort(first_second_third_fourth_relations.columns),
         axis=1)
)
first_second_third_fourth_relations['MedDRA_concept_id_4'] = \
first_second_third_fourth_relations['MedDRA_concept_id_4'].astype(int)
print(first_second_third_fourth_relations.shape)
print(first_second_third_fourth_relations.head())
print(first_second_third_fourth_relations.MedDRA_concept_class_id_1.value_counts())
print(first_second_third_fourth_relations.MedDRA_concept_class_id_2.value_counts())
print(first_second_third_fourth_relations.MedDRA_concept_class_id_3.value_counts())
print(first_second_third_fourth_relations.MedDRA_concept_class_id_4.value_counts())

(927, 19)
  MedDRA_concept_class_id_1 MedDRA_concept_class_id_2  \
0                        PT                       HLT   
1                        PT                       HLT   
2                        PT                       HLT   
3                        PT                       HLT   
4                        PT                       HLT   

  MedDRA_concept_class_id_3 MedDRA_concept_class_id_4  MedDRA_concept_code_1  \
0                      HLGT                       SOC               10053745   
1                      HLGT                       SOC               10056496   
2                      HLGT                       SOC               10068370   
3                      HLGT                       SOC               10069495   
4                      HLGT                       SOC               10074561   

   MedDRA_concept_code_2  MedDRA_concept_code_3  MedDRA_concept_code_4  \
0               10009737               10064477               10005329   
1               10

In [58]:
len(np.setdiff1d(reactions,first_second_third_fourth_relations.MedDRA_concept_id_1.unique()))

1

In [59]:
left_over = np.setdiff1d(reactions,first_second_third_fourth_relations.MedDRA_concept_id_1.unique())

all_meddra_relationships.query('MedDRA_concept_id_1 in @left_over')

,MedDRA_concept_class_id_1,MedDRA_concept_class_id_2,MedDRA_concept_code_1,MedDRA_concept_code_2,MedDRA_concept_id_1,MedDRA_concept_id_2,MedDRA_concept_name_1,MedDRA_concept_name_2,relationship_id_12


In [60]:
df1 = (standard_reactions.
       loc[:,['MedDRA_concept_id']].
       drop_duplicates().
       dropna().
       set_index('MedDRA_concept_id')
      )
print(df1.shape)

(542, 0)


In [61]:
df2 = (first_second_third_fourth_relations.
       set_index('MedDRA_concept_id_1')
      )
print(df2.shape)

(927, 18)


In [62]:
joined = df1.join(df2).rename_axis('MedDRA_concept_id_1').reset_index().dropna()
joined = joined.reindex(np.sort(joined.columns),axis=1)
joined.MedDRA_concept_id_1 = joined.MedDRA_concept_id_1.astype(int).copy()
joined.MedDRA_concept_id_2 = joined.MedDRA_concept_id_2.astype(int).copy()
joined.MedDRA_concept_id_3 = joined.MedDRA_concept_id_3.astype(int).copy()
joined.MedDRA_concept_id_4 = joined.MedDRA_concept_id_4.astype(int).copy()
joined.MedDRA_concept_code_1 = joined.MedDRA_concept_code_1.astype(int).copy()
joined.MedDRA_concept_code_2 = joined.MedDRA_concept_code_2.astype(int).copy()
joined.MedDRA_concept_code_3 = joined.MedDRA_concept_code_3.astype(int).copy()
joined.MedDRA_concept_code_4 = joined.MedDRA_concept_code_4.astype(int).copy()
print(joined.shape)
print(joined.head())

(927, 19)
  MedDRA_concept_class_id_1 MedDRA_concept_class_id_2  \
0                        PT                       HLT   
1                        PT                       HLT   
2                        PT                       HLT   
3                        PT                       HLT   
4                        PT                       HLT   

  MedDRA_concept_class_id_3 MedDRA_concept_class_id_4  MedDRA_concept_code_1  \
0                      HLGT                       SOC               10077983   
1                      HLGT                       SOC               10078057   
2                      HLGT                       SOC               10078057   
3                      HLGT                       SOC               10077305   
4                      HLGT                       SOC               10077466   

   MedDRA_concept_code_2  MedDRA_concept_code_3  MedDRA_concept_code_4  \
0               10052777               10000073               10017947   
1               10

In [63]:
print(joined.MedDRA_concept_class_id_1.value_counts())
print(joined.MedDRA_concept_class_id_2.value_counts())
print(joined.MedDRA_concept_class_id_3.value_counts())
print(joined.MedDRA_concept_class_id_4.value_counts())

PT    927
Name: MedDRA_concept_class_id_1, dtype: int64
HLT    927
Name: MedDRA_concept_class_id_2, dtype: int64
HLGT    927
Name: MedDRA_concept_class_id_3, dtype: int64
SOC    927
Name: MedDRA_concept_class_id_4, dtype: int64


In [23]:
(joined.
 to_csv(er_dir+'standard_reactions_meddra_relationships.csv.gz',
        compression='gzip',index=False)
)

In [64]:
pt_to_soc = (joined.
loc[:,['MedDRA_concept_id_1','MedDRA_concept_code_1',
       'MedDRA_concept_name_1','MedDRA_concept_class_id_1',
       'MedDRA_concept_id_4','MedDRA_concept_code_4',
       'MedDRA_concept_name_4','MedDRA_concept_class_id_4']].
query('MedDRA_concept_class_id_4=="SOC"').
              drop_duplicates()
)
print(pt_to_soc.shape)
print(pt_to_soc.head())

(920, 8)
   MedDRA_concept_id_1  MedDRA_concept_code_1  \
0               788109               10077983   
1               788129               10078057   
2               788129               10078057   
3               788229               10077305   
4               788295               10077466   

                               MedDRA_concept_name_1  \
0                                Abdominal fat apron   
1  Adenoid cystic carcinoma of external auditory ...   
2  Adenoid cystic carcinoma of external auditory ...   
3                     Acute on chronic liver failure   
4                         Adventitial cystic disease   

  MedDRA_concept_class_id_1  MedDRA_concept_id_4  MedDRA_concept_code_4  \
0                        PT             35700000               10017947   
1                        PT             35400000               10013993   
2                        PT             36600000               10029104   
3                        PT             35900000           

In [65]:
pt_to_hlgt = (joined.
loc[:,['MedDRA_concept_id_1','MedDRA_concept_code_1',
       'MedDRA_concept_name_1','MedDRA_concept_class_id_1',
       'MedDRA_concept_id_3','MedDRA_concept_code_3',
       'MedDRA_concept_name_3','MedDRA_concept_class_id_3']].
query('MedDRA_concept_class_id_3=="HLGT"').
              drop_duplicates()
)
print(pt_to_hlgt.shape)
print(pt_to_hlgt.head())

(823, 8)
   MedDRA_concept_id_1  MedDRA_concept_code_1  \
0               788109               10077983   
1               788129               10078057   
2               788129               10078057   
3               788229               10077305   
4               788295               10077466   

                               MedDRA_concept_name_1  \
0                                Abdominal fat apron   
1  Adenoid cystic carcinoma of external auditory ...   
2  Adenoid cystic carcinoma of external auditory ...   
3                     Acute on chronic liver failure   
4                         Adventitial cystic disease   

  MedDRA_concept_class_id_1  MedDRA_concept_id_3  MedDRA_concept_code_3  \
0                        PT             35702107               10000073   
1                        PT             35402081               10015732   
2                        PT             36602218               10027655   
3                        PT             35902137           

In [66]:
pt_to_hlt = (joined.
loc[:,['MedDRA_concept_id_1','MedDRA_concept_code_1',
       'MedDRA_concept_name_1','MedDRA_concept_class_id_1',
       'MedDRA_concept_id_2','MedDRA_concept_code_2',
       'MedDRA_concept_name_2','MedDRA_concept_class_id_2']].
query('MedDRA_concept_class_id_2=="HLT"').
             drop_duplicates()
)
print(pt_to_hlt.shape)
print(pt_to_hlt.head())

(824, 8)
   MedDRA_concept_id_1  MedDRA_concept_code_1  \
0               788109               10077983   
1               788129               10078057   
2               788129               10078057   
3               788229               10077305   
4               788295               10077466   

                               MedDRA_concept_name_1  \
0                                Abdominal fat apron   
1  Adenoid cystic carcinoma of external auditory ...   
2  Adenoid cystic carcinoma of external auditory ...   
3                     Acute on chronic liver failure   
4                         Adventitial cystic disease   

  MedDRA_concept_class_id_1  MedDRA_concept_id_2  MedDRA_concept_code_2  \
0                        PT             35702709               10052777   
1                        PT             35402591               10027672   
2                        PT             36603346               10003793   
3                        PT             35902857           

In [67]:
standard_reactions_pt_to_hlt = \
(standard_reactions.
 loc[:,['safetyreportid','MedDRA_concept_id']].
 drop_duplicates().
 set_index(['MedDRA_concept_id']).
 join(pt_to_hlt.
      loc[:,['MedDRA_concept_id_1','MedDRA_concept_id_2',
           'MedDRA_concept_code_2','MedDRA_concept_name_2',
           'MedDRA_concept_class_id_2']].
      set_index('MedDRA_concept_id_1')
     ).
 reset_index(drop=True).
 rename(
     columns={
         'MedDRA_concept_id_2' : 'MedDRA_concept_id',
         'MedDRA_concept_code_2' : 'MedDRA_concept_code',
         'MedDRA_concept_name_2' : 'MedDRA_concept_name',
         'MedDRA_concept_class_id_2' : 'MedDRA_concept_class_id'
     }
 ).
 dropna().
 drop_duplicates()
)
standard_reactions_pt_to_hlt = (standard_reactions_pt_to_hlt.
                               reindex(np.sort(standard_reactions_pt_to_hlt.columns),axis=1)
                               )
print(standard_reactions_pt_to_hlt.shape)
print(standard_reactions_pt_to_hlt.head())

(1152814, 5)
  safetyreportid  MedDRA_concept_id  MedDRA_concept_code  \
0       10361172         35702709.0           10052777.0   
1       11817798         35702709.0           10052777.0   
2       12319163         35702709.0           10052777.0   
3       12688362         35702709.0           10052777.0   
4       12801872         35702709.0           10052777.0   

             MedDRA_concept_name MedDRA_concept_class_id  
0  Abdominal wall conditions NEC                     HLT  
1  Abdominal wall conditions NEC                     HLT  
2  Abdominal wall conditions NEC                     HLT  
3  Abdominal wall conditions NEC                     HLT  
4  Abdominal wall conditions NEC                     HLT  


In [68]:
print(
    len(
        np.intersect1d(
            all_reports,
            standard_reactions_pt_to_hlt.safetyreportid.astype(str).unique()
                        )
    )/len(all_reports)
)

0.9956864990966461


In [29]:
(standard_reactions_pt_to_hlt.
 to_csv(er_dir+'standard_reactions_meddra_hlt.csv.gz',
        compression='gzip',index=False)
)

In [69]:
standard_reactions_pt_to_hlgt = \
(standard_reactions.
 loc[:,['safetyreportid','MedDRA_concept_id']].
 drop_duplicates().
 set_index(['MedDRA_concept_id']).
 join(pt_to_hlgt.
      loc[:,['MedDRA_concept_id_1','MedDRA_concept_id_3',
           'MedDRA_concept_code_3','MedDRA_concept_name_3',
           'MedDRA_concept_class_id_3']].
      set_index('MedDRA_concept_id_1')
     ).
 reset_index(drop=True).
 rename(
     columns={
         'MedDRA_concept_id_3' : 'MedDRA_concept_id',
         'MedDRA_concept_code_3' : 'MedDRA_concept_code',
         'MedDRA_concept_name_3' : 'MedDRA_concept_name',
         'MedDRA_concept_class_id_3' : 'MedDRA_concept_class_id'
     }
 ).
 dropna().
 drop_duplicates()
)
standard_reactions_pt_to_hlgt = (standard_reactions_pt_to_hlgt.
                               reindex(np.sort(standard_reactions_pt_to_hlgt.columns),axis=1)
                               )
print(standard_reactions_pt_to_hlgt.shape)
print(standard_reactions_pt_to_hlgt.head())

(1099874, 5)
  safetyreportid  MedDRA_concept_id  MedDRA_concept_code  \
0       10361172         35702107.0           10000073.0   
1       11817798         35702107.0           10000073.0   
2       12319163         35702107.0           10000073.0   
3       12688362         35702107.0           10000073.0   
4       12801872         35702107.0           10000073.0   

                                 MedDRA_concept_name MedDRA_concept_class_id  
0  Abdominal hernias and other abdominal wall con...                    HLGT  
1  Abdominal hernias and other abdominal wall con...                    HLGT  
2  Abdominal hernias and other abdominal wall con...                    HLGT  
3  Abdominal hernias and other abdominal wall con...                    HLGT  
4  Abdominal hernias and other abdominal wall con...                    HLGT  


In [70]:
print(
    len(
        np.intersect1d(
            all_reports,
            standard_reactions_pt_to_hlgt.safetyreportid.astype(str).unique()
                        )
    )/len(all_reports)
)

0.9956864990966461


In [32]:
(standard_reactions_pt_to_hlgt.
 to_csv(er_dir+'standard_reactions_meddra_hlgt.csv.gz',
        compression='gzip',index=False)
)

In [71]:
standard_reactions_pt_to_soc = \
(standard_reactions.
 loc[:,['safetyreportid','MedDRA_concept_id']].
 drop_duplicates().
 set_index(['MedDRA_concept_id']).
 join(pt_to_soc.
      loc[:,['MedDRA_concept_id_1','MedDRA_concept_id_4',
           'MedDRA_concept_code_4','MedDRA_concept_name_4',
           'MedDRA_concept_class_id_4']].
      set_index('MedDRA_concept_id_1')
     ).
 reset_index(drop=True).
 rename(
     columns={
         'MedDRA_concept_id_4' : 'MedDRA_concept_id',
         'MedDRA_concept_code_4' : 'MedDRA_concept_code',
         'MedDRA_concept_name_4' : 'MedDRA_concept_name',
         'MedDRA_concept_class_id_4' : 'MedDRA_concept_class_id'
     }
 ).
 dropna().
 drop_duplicates()
)
standard_reactions_pt_to_soc = (standard_reactions_pt_to_soc.
                               reindex(np.sort(standard_reactions_pt_to_soc.columns),axis=1)
                               )
print(standard_reactions_pt_to_soc.shape)
print(standard_reactions_pt_to_soc.head())

(1101829, 5)
  safetyreportid  MedDRA_concept_id  MedDRA_concept_code  \
0       10361172         35700000.0           10017947.0   
1       11817798         35700000.0           10017947.0   
2       12319163         35700000.0           10017947.0   
3       12688362         35700000.0           10017947.0   
4       12801872         35700000.0           10017947.0   

          MedDRA_concept_name MedDRA_concept_class_id  
0  Gastrointestinal disorders                     SOC  
1  Gastrointestinal disorders                     SOC  
2  Gastrointestinal disorders                     SOC  
3  Gastrointestinal disorders                     SOC  
4  Gastrointestinal disorders                     SOC  


In [72]:
print(
    len(
        np.intersect1d(
            all_reports,
            standard_reactions_pt_to_soc.safetyreportid.astype(str).unique()
                        )
    )/len(all_reports)
)

0.9956864990966461


In [35]:
(standard_reactions_pt_to_soc.
 to_csv(er_dir+'standard_reactions_meddra_soc.csv.gz',
        compression='gzip',index=False)
)

In [73]:
del c
del r
del first_relations
del second_relations
del first_second_third_relations
del all_meddra_relationships
del meddra_concept
del df1
del df2
del joined
del standard_reactions_pt_to_soc
del standard_reactions_pt_to_hlgt
del standard_reactions_pt_to_hlt

### standard_reactions_snomed

In [86]:
standard_reactions_meddra_relationships = (pd.read_csv(
    er_dir+'standard_reactions_meddra_relationships.csv.gz',
    compression='gzip',
    dtype={
    'safetyreportid' : 'str'
    })
    )

print(standard_reactions_meddra_relationships.MedDRA_concept_id_1.nunique())
print(standard_reactions_meddra_relationships.MedDRA_concept_id_2.nunique())
print(standard_reactions_meddra_relationships.MedDRA_concept_id_3.nunique())
print(standard_reactions_meddra_relationships.MedDRA_concept_id_4.nunique())

standard_reactions_meddra_relationships.MedDRA_concept_id_1 = \
standard_reactions_meddra_relationships.MedDRA_concept_id_1.astype(int)

standard_reactions_meddra_relationships.MedDRA_concept_id_2 = \
standard_reactions_meddra_relationships.MedDRA_concept_id_2.astype(int)

standard_reactions_meddra_relationships.MedDRA_concept_id_3 = \
standard_reactions_meddra_relationships.MedDRA_concept_id_3.astype(int)

standard_reactions_meddra_relationships.MedDRA_concept_id_4 = \
standard_reactions_meddra_relationships.MedDRA_concept_id_4.astype(int)

print(standard_reactions_meddra_relationships.shape)
print(standard_reactions_meddra_relationships.head())

18653
1736
338
27
(29381, 19)
  MedDRA_concept_class_id_1 MedDRA_concept_class_id_2  \
0                        PT                       HLT   
1                        PT                       HLT   
2                        PT                       HLT   
3                        PT                       HLT   
4                        PT                       HLT   

  MedDRA_concept_class_id_3 MedDRA_concept_class_id_4  MedDRA_concept_code_1  \
0                      HLGT                       SOC               10077233   
1                      HLGT                       SOC               10077234   
2                      HLGT                       SOC               10077237   
3                      HLGT                       SOC               10077238   
4                      HLGT                       SOC               10077243   

   MedDRA_concept_code_2  MedDRA_concept_code_3  MedDRA_concept_code_4  \
0               10005037               10004994               10038359  

In [87]:
reactions = standard_reactions_meddra_relationships.MedDRA_concept_id_1.unique()
print(len(reactions))
meddra_concept_ids = concept.query('vocabulary_id=="MedDRA"').concept_id.astype(int).unique()
len(meddra_concept_ids)

intersect = np.intersect1d(reactions,meddra_concept_ids)
print(len(intersect))
print(len(intersect)/len(reactions))

18653
18653
1.0


In [88]:
m_to_s_r = (concept_relationship.
            query('relationship_id=="MedDRA - SNOMED eq"').
            loc[:,['concept_id_1','concept_id_2']].
            drop_duplicates().
            set_index('concept_id_2').
            join(concept.
                 query('vocabulary_id=="SNOMED"').
                 loc[:,['concept_id','concept_code','concept_class_id','concept_name']].
                 drop_duplicates().
                 set_index('concept_id')
                ).
            rename_axis('SNOMED_concept_id').
            reset_index().
            rename(columns={
                'concept_id_1' : 'MedDRA_concept_id',
                'concept_name' : 'SNOMED_concept_name',
                'concept_code' : 'SNOMED_concept_code',
                'concept_class_id' : 'SNOMED_concept_class_id'
            })
)
m_to_s_r.MedDRA_concept_id = m_to_s_r.MedDRA_concept_id.astype(int)
m_to_s_r = m_to_s_r.reindex(np.sort(m_to_s_r.columns),axis=1)
print(m_to_s_r.shape)
print(m_to_s_r.SNOMED_concept_class_id.value_counts())
print(m_to_s_r.head())

(13633, 5)
Clinical Finding     11933
Procedure             1610
Context-dependent       74
Event                    7
Morph Abnormality        4
Physical Force           3
Specimen                 1
Body Structure           1
Name: SNOMED_concept_class_id, dtype: int64
   MedDRA_concept_id SNOMED_concept_class_id SNOMED_concept_code  \
0           36617788        Clinical Finding            94899001   
1           35104270        Clinical Finding           127040003   
2           35104702        Clinical Finding           191169008   
3           35104311        Clinical Finding           191169008   
4           35708362        Clinical Finding            14223005   

   SNOMED_concept_id                       SNOMED_concept_name  
0              22274  Neoplasm of uncertain behavior of larynx  
1              22281         Sickle cell-hemoglobin SS disease  
2              22288                 Hereditary elliptocytosis  
3              22288                 Hereditary elliptocytos

In [89]:
r2s = m_to_s_r.MedDRA_concept_id.unique()

In [90]:
pts = (standard_reactions_meddra_relationships.
       query('MedDRA_concept_class_id_1=="PT"').
       MedDRA_concept_id_1.
       unique())
print(len(np.intersect1d(pts,r2s))/len(pts))
print(len(np.intersect1d(pts,r2s))/len(r2s))

df = (standard_reactions_meddra_relationships.
      query('MedDRA_concept_id_1 in @r2s'))

print(df.shape)

joinedpt = (df.
           set_index('MedDRA_concept_id_1').
           join(m_to_s_r.
                query('MedDRA_concept_id in @pts').
                set_index('MedDRA_concept_id')
               ).
           rename_axis('MedDRA_concept_id_1').
           reset_index().
           rename(columns={
               'SNOMED_concept_id' : 'SNOMED_concept_id_1',
               'SNOMED_concept_code' : 'SNOMED_concept_code_1',
               'SNOMED_concept_name' : 'SNOMED_concept_name_1',
               'SNOMED_concept_class_id' : 'SNOMED_concept_class_id_1',
           }).
           dropna()
          )
joinedpt = joinedpt.reindex(np.sort(joinedpt.columns),axis=1)
print(joinedpt.shape)
print(joinedpt.head())

0.45504744545113385
0.8758642039005262
(13699, 19)
(19901, 23)
  MedDRA_concept_class_id_1 MedDRA_concept_class_id_2  \
0                        PT                       HLT   
1                        PT                       HLT   
2                        PT                       HLT   
3                        PT                       HLT   
4                        PT                       HLT   

  MedDRA_concept_class_id_3 MedDRA_concept_class_id_4  MedDRA_concept_code_1  \
0                      HLGT                       SOC               10002043   
1                      HLGT                       SOC               10002043   
2                      HLGT                       SOC               10066468   
3                      HLGT                       SOC               10066468   
4                      HLGT                       SOC               10066468   

   MedDRA_concept_code_2  MedDRA_concept_code_3  MedDRA_concept_code_4  \
0               10002042               

In [91]:
hlts = (joinedpt.
       query('MedDRA_concept_class_id_2=="HLT"').
       MedDRA_concept_id_2.
       unique())
print(len(np.intersect1d(hlts,r2s))/len(hlts))
print(len(np.intersect1d(hlts,r2s))/len(r2s))

df = (joinedpt.copy())

print(df.shape)
print(df.head())


joinedhlt = (df.
           set_index('MedDRA_concept_id_2').
           join(m_to_s_r.
                query('MedDRA_concept_id in @hlts').
                set_index('MedDRA_concept_id')
               ).
           rename_axis('MedDRA_concept_id_2').
           reset_index().
           rename(columns={
               'SNOMED_concept_id' : 'SNOMED_concept_id_2',
               'SNOMED_concept_code' : 'SNOMED_concept_code_2',
               'SNOMED_concept_name' : 'SNOMED_concept_name_2',
               'SNOMED_concept_class_id' : 'SNOMED_concept_class_id_2',
           })
          )
joinedhlt = joinedhlt.reindex(np.sort(joinedhlt.columns),axis=1)
print(joinedhlt.shape)
print(joinedhlt.head())

0.025355596784168214
0.004230729542874833
(19901, 23)
  MedDRA_concept_class_id_1 MedDRA_concept_class_id_2  \
0                        PT                       HLT   
1                        PT                       HLT   
2                        PT                       HLT   
3                        PT                       HLT   
4                        PT                       HLT   

  MedDRA_concept_class_id_3 MedDRA_concept_class_id_4  MedDRA_concept_code_1  \
0                      HLGT                       SOC               10002043   
1                      HLGT                       SOC               10002043   
2                      HLGT                       SOC               10066468   
3                      HLGT                       SOC               10066468   
4                      HLGT                       SOC               10066468   

   MedDRA_concept_code_2  MedDRA_concept_code_3  MedDRA_concept_code_4  \
0               10002042               10002086 

In [92]:
hlgts = (joinedhlt.
       query('MedDRA_concept_class_id_3=="HLGT"').
       MedDRA_concept_id_3.
       unique())
print(len(np.intersect1d(hlgts,r2s))/len(hlgts))
print(len(np.intersect1d(hlgts,r2s))/len(r2s))

df = (joinedhlt.copy())

print(df.shape)

joinedhlgt = (df.
           set_index('MedDRA_concept_id_3').
           join(m_to_s_r.
                query('MedDRA_concept_id in @hlgts').
                set_index('MedDRA_concept_id')
               ).
           rename_axis('MedDRA_concept_id_3').
           reset_index().
           drop_duplicates().
           rename(columns={
               'SNOMED_concept_id' : 'SNOMED_concept_id_3',
               'SNOMED_concept_code' : 'SNOMED_concept_code_3',
               'SNOMED_concept_name' : 'SNOMED_concept_name_3',
               'SNOMED_concept_class_id' : 'SNOMED_concept_class_id_3',
           })
          )
joinedhlgt = joinedhlgt.reindex(np.sort(joinedhlgt.columns),axis=1)
print(joinedhlgt.shape)
print(joinedhlgt.head())

0.03571428571428571
0.0012382623052316583
(20338, 27)
(20670, 31)
  MedDRA_concept_class_id_1 MedDRA_concept_class_id_2  \
0                        PT                       HLT   
1                        PT                       HLT   
2                        PT                       HLT   
3                        PT                       HLT   
4                        PT                       HLT   

  MedDRA_concept_class_id_3 MedDRA_concept_class_id_4  MedDRA_concept_code_1  \
0                      HLGT                       SOC               10005179   
1                      HLGT                       SOC               10023003   
2                      HLGT                       SOC               10047709   
3                      HLGT                       SOC               10036069   
4                      HLGT                       SOC               10002954   

   MedDRA_concept_code_2  MedDRA_concept_code_3  MedDRA_concept_code_4  \
0               10077547            

In [93]:
socs = (joinedhlgt.
       query('MedDRA_concept_class_id_4=="SOC"').
       MedDRA_concept_id_4.
       unique())
print(len(np.intersect1d(socs,r2s))/len(socs))
print(len(np.intersect1d(socs,r2s))/len(r2s))

df = (joinedhlgt.copy())

print(df.shape)
print(df.head())
print(m_to_s_r.shape)
print(m_to_s_r.head())

joinedsoc = (df.
           set_index('MedDRA_concept_id_4').
           join(m_to_s_r.
                query('MedDRA_concept_id in @socs').
                set_index('MedDRA_concept_id')
               ).
           rename_axis('MedDRA_concept_id_4').
           reset_index().
           drop_duplicates().
           rename(columns={
               'SNOMED_concept_id' : 'SNOMED_concept_id_4',
               'SNOMED_concept_code' : 'SNOMED_concept_code_4',
               'SNOMED_concept_name' : 'SNOMED_concept_name_4',
               'SNOMED_concept_class_id' : 'SNOMED_concept_class_id_4',
           })
          )
joinedsoc = joinedsoc.reindex(np.sort(joinedsoc.columns),axis=1)
print(joinedsoc.shape)
print(joinedsoc.head())

0.0
0.0
(20670, 31)
  MedDRA_concept_class_id_1 MedDRA_concept_class_id_2  \
0                        PT                       HLT   
1                        PT                       HLT   
2                        PT                       HLT   
3                        PT                       HLT   
4                        PT                       HLT   

  MedDRA_concept_class_id_3 MedDRA_concept_class_id_4  MedDRA_concept_code_1  \
0                      HLGT                       SOC               10005179   
1                      HLGT                       SOC               10023003   
2                      HLGT                       SOC               10047709   
3                      HLGT                       SOC               10036069   
4                      HLGT                       SOC               10002954   

   MedDRA_concept_code_2  MedDRA_concept_code_3  MedDRA_concept_code_4  \
0               10077547               10077546               10037175   
1       

In [94]:
smeddraconcepts = joinedpt.MedDRA_concept_id_1.unique()
print(len(smeddraconcepts))
allmeddraconcepts = (standard_reactions_meddra_relationships.
                     query('MedDRA_concept_class_id_1=="PT"').
                     MedDRA_concept_id_1.
                     unique())
print(len(allmeddraconcepts))

print(len(np.intersect1d(smeddraconcepts,allmeddraconcepts))/len(smeddraconcepts))
print(len(np.intersect1d(smeddraconcepts,allmeddraconcepts))/len(allmeddraconcepts))

8488
18653
1.0
0.45504744545113385


In [95]:
smeddraconcepts = joinedhlt.MedDRA_concept_id_2.unique()
print(len(smeddraconcepts))
allmeddraconcepts = (standard_reactions_meddra_relationships.
                     query('MedDRA_concept_class_id_2=="HLT"').
                     MedDRA_concept_id_2.
                     unique())
print(len(allmeddraconcepts))

print(len(np.intersect1d(smeddraconcepts,allmeddraconcepts))/len(smeddraconcepts))
print(len(np.intersect1d(smeddraconcepts,allmeddraconcepts))/len(allmeddraconcepts))

1617
1736
1.0
0.9314516129032258


In [96]:
smeddraconcepts = joinedhlgt.MedDRA_concept_id_3.unique()
print(len(smeddraconcepts))
allmeddraconcepts = (standard_reactions_meddra_relationships.
                     query('MedDRA_concept_class_id_3=="HLGT"').
                     MedDRA_concept_id_3.
                     unique())
print(len(allmeddraconcepts))

print(len(np.intersect1d(smeddraconcepts,allmeddraconcepts))/len(smeddraconcepts))
print(len(np.intersect1d(smeddraconcepts,allmeddraconcepts))/len(allmeddraconcepts))

336
338
1.0
0.9940828402366864


In [97]:
smeddraconcepts = joinedsoc.MedDRA_concept_id_4.unique()
print(len(smeddraconcepts))
allmeddraconcepts = (standard_reactions_meddra_relationships.
                     query('MedDRA_concept_class_id_4=="SOC"').
                     MedDRA_concept_id_4.
                     unique())
print(len(allmeddraconcepts))

print(len(np.intersect1d(smeddraconcepts,allmeddraconcepts))/len(smeddraconcepts))
print(len(np.intersect1d(smeddraconcepts,allmeddraconcepts))/len(allmeddraconcepts))

27
27
1.0
1.0


In [98]:
print(joinedsoc.head())
print(joinedsoc.shape)
print(joinedsoc[joinedsoc.SNOMED_concept_id_1.notnull()].shape)
print(joinedsoc.SNOMED_concept_id_1.nunique())
print(joinedsoc[joinedsoc.SNOMED_concept_id_2.notnull()].shape)
print(joinedsoc.SNOMED_concept_id_2.nunique())
print(joinedsoc[joinedsoc.SNOMED_concept_id_3.notnull()].shape)
print(joinedsoc.SNOMED_concept_id_3.nunique())
print(joinedsoc[joinedsoc.SNOMED_concept_id_4.notnull()].shape)
print(joinedsoc.SNOMED_concept_id_4.nunique())

  MedDRA_concept_class_id_1 MedDRA_concept_class_id_2  \
0                        PT                       HLT   
1                        PT                       HLT   
2                        PT                       HLT   
3                        PT                       HLT   
4                        PT                       HLT   

  MedDRA_concept_class_id_3 MedDRA_concept_class_id_4  MedDRA_concept_code_1  \
0                      HLGT                       SOC               10005179   
1                      HLGT                       SOC               10023003   
2                      HLGT                       SOC               10047709   
3                      HLGT                       SOC               10036069   
4                      HLGT                       SOC               10002954   

   MedDRA_concept_code_2  MedDRA_concept_code_3  MedDRA_concept_code_4  \
0               10077547               10077546               10037175   
1               10077547    

In [99]:
joinedsoc.SNOMED_concept_code_1 = joinedsoc.SNOMED_concept_code_1.astype(int)
joinedsoc.SNOMED_concept_code_2 = joinedsoc.SNOMED_concept_code_2.astype(float)
joinedsoc.SNOMED_concept_code_3 = joinedsoc.SNOMED_concept_code_3.astype(float)
joinedsoc.SNOMED_concept_code_4 = joinedsoc.SNOMED_concept_code_4.astype(float)

In [105]:
standard_reactions = (pd.
                      read_csv(er_dir+'standard_reactions.csv.gz',
                               compression="gzip",
                              dtype={
                                  'safetyreportid' : 'str'
                              })
                     )
all_reports = (standard_reactions.safetyreportid.unique())
print(standard_reactions.shape)
print(standard_reactions.head())

(1000000, 6)
              MedDRA_concept_name safetyreportid    reaction_outcome  \
0  11-Beta-Hydroxylase Deficiency      7999037-5                 NaN   
1  17 Ketosteroids Urine Abnormal       13978283  Recovered/resolved   
2  17 Ketosteroids Urine Abnormal       13983944  Recovered/resolved   
3  17 Ketosteroids Urine Abnormal       13988153  Recovered/resolved   
4  17 Ketosteroids Urine Abnormal       13993404  Recovered/resolved   

   MedDRA_concept_id  MedDRA_concept_code MedDRA_concept_class_id  
0           35305836             10000002                      PT  
1           36312356             10061608                      PT  
2           36312356             10061608                      PT  
3           36312356             10061608                      PT  
4           36312356             10061608                      PT  


In [106]:
standard_reactions_meddrapt_to_snomed = \
(joinedsoc.
 loc[:,['MedDRA_concept_id_1','SNOMED_concept_id_1',
   'SNOMED_concept_code_1','SNOMED_concept_name_1',
   'SNOMED_concept_class_id_1']].
 drop_duplicates().
 rename(
     columns={
         'SNOMED_concept_id_1' : 'SNOMED_concept_id',
         'SNOMED_concept_code_1' : 'SNOMED_concept_code',
         'SNOMED_concept_name_1' : 'SNOMED_concept_name',
         'SNOMED_concept_class_id_1' : 'SNOMED_concept_class_id'
     }
 ).
 set_index('MedDRA_concept_id_1').
 join(standard_reactions.
      drop_duplicates().
      set_index('MedDRA_concept_id')
     ).
 reset_index(drop=True).
 drop(['MedDRA_concept_code','MedDRA_concept_name',
      'MedDRA_concept_class_id'],axis=1).
 dropna()
)
standard_reactions_meddrapt_to_snomed = \
(standard_reactions_meddrapt_to_snomed.
 reindex(np.sort(standard_reactions_meddrapt_to_snomed.columns),
         axis=1))
print(standard_reactions_meddrapt_to_snomed.shape)
print(standard_reactions_meddrapt_to_snomed.head())

(1114406, 6)
   SNOMED_concept_class_id  SNOMED_concept_code  SNOMED_concept_id  \
72        Clinical Finding            421527008            4225324   
73        Clinical Finding            421527008            4225324   
74        Clinical Finding            421527008            4225324   
75        Clinical Finding            421527008            4225324   
76        Clinical Finding            421527008            4225324   

                                  SNOMED_concept_name  \
72  Resistance to activated protein C due to Facto...   
73  Resistance to activated protein C due to Facto...   
74  Resistance to activated protein C due to Facto...   
75  Resistance to activated protein C due to Facto...   
76  Resistance to activated protein C due to Facto...   

              reaction_outcome safetyreportid  
72                     Unknown       10509510  
73                         NaN       12225991  
74  Not recovered/not resolved       12638004  
75                     Unknown 

In [107]:
print(
    len(
        np.intersect1d(
            all_reports,
            standard_reactions_meddrapt_to_snomed.safetyreportid.astype(str).unique()
        )
    )/len(all_reports)
)

0.8028218315873661


In [22]:
(standard_reactions_meddrapt_to_snomed.
 to_csv(er_dir+'standard_reactions_snomed.csv.gz',
        compression='gzip',index=False)
)

In [103]:
standard_reactions_meddrahlt_to_snomed = \
(joinedsoc.
 query('MedDRA_concept_class_id_2=="HLT"').
 loc[:,['MedDRA_concept_id_1','SNOMED_concept_id_2',
   'SNOMED_concept_code_2','SNOMED_concept_name_2',
   'SNOMED_concept_class_id_2']].
 drop_duplicates().
 rename(
     columns={
         'SNOMED_concept_id_2' : 'SNOMED_concept_id',
         'SNOMED_concept_code_2' : 'SNOMED_concept_code',
         'SNOMED_concept_name_2' : 'SNOMED_concept_name',
         'SNOMED_concept_class_id_2' : 'SNOMED_concept_class_id'
     }
 ).
 set_index('MedDRA_concept_id_1').
 join(standard_reactions.
      drop_duplicates().
      set_index('MedDRA_concept_id')
     ).
 rename_axis('MedDRA_concept_id').
 reset_index().
 dropna(subset=['MedDRA_concept_id','SNOMED_concept_id','safetyreportid'])
)
standard_reactions_meddrahlt_to_snomed = \
(standard_reactions_meddrahlt_to_snomed.
 reindex(np.sort(standard_reactions_meddrahlt_to_snomed.columns),
         axis=1))
print(standard_reactions_meddrahlt_to_snomed.shape)
print(standard_reactions_meddrahlt_to_snomed.head())

(0, 10)
Empty DataFrame
Columns: [MedDRA_concept_class_id, MedDRA_concept_code, MedDRA_concept_id, MedDRA_concept_name, SNOMED_concept_class_id, SNOMED_concept_code, SNOMED_concept_id, SNOMED_concept_name, reaction_outcome, safetyreportid]
Index: []


In [24]:
print(
    len(
        np.intersect1d(
            all_reports,
            standard_reactions_meddrahlt_to_snomed.safetyreportid.astype(str).unique()
        )
    )/len(all_reports)
)

0.0


In [25]:
standard_reactions_meddrahlgt_to_snomed = \
(joinedsoc.
 query('MedDRA_concept_class_id_2=="HLGT"').
 loc[:,['MedDRA_concept_id_1','SNOMED_concept_id_3',
   'SNOMED_concept_code_3','SNOMED_concept_name_3',
   'SNOMED_concept_class_id_2']].
 drop_duplicates().
 rename(
     columns={
         'SNOMED_concept_id_3' : 'SNOMED_concept_id',
         'SNOMED_concept_code_3' : 'SNOMED_concept_code',
         'SNOMED_concept_name_3' : 'SNOMED_concept_name',
         'SNOMED_concept_class_id_3' : 'SNOMED_concept_class_id'
     }
 ).
 set_index('MedDRA_concept_id_1').
 join(standard_reactions.
      drop_duplicates().
      set_index('MedDRA_concept_id')
     ).
 rename_axis('MedDRA_concept_id').
 reset_index().
 dropna(subset=['MedDRA_concept_id','SNOMED_concept_id','safetyreportid'])
)
standard_reactions_meddrahlgt_to_snomed = \
(standard_reactions_meddrahlgt_to_snomed.
 reindex(np.sort(standard_reactions_meddrahlgt_to_snomed.columns),
         axis=1))
print(standard_reactions_meddrahlgt_to_snomed.shape)
print(standard_reactions_meddrahlgt_to_snomed.head())

(0, 10)
Empty DataFrame
Columns: [MedDRA_concept_class_id, MedDRA_concept_code, MedDRA_concept_id, MedDRA_concept_name, SNOMED_concept_class_id_2, SNOMED_concept_code, SNOMED_concept_id, SNOMED_concept_name, reaction_outcome, safetyreportid]
Index: []


In [26]:
print(
    len(
        np.intersect1d(
            all_reports,
            standard_reactions_meddrahlgt_to_snomed.safetyreportid.astype(str).unique()
        )
    )/len(all_reports)
)

0.0


In [27]:
standard_reactions_meddrasoc_to_snomed = \
(joinedsoc.
 query('MedDRA_concept_class_id_4=="SOC"').
 loc[:,['MedDRA_concept_id_1','SNOMED_concept_id_4',
   'SNOMED_concept_code_4','SNOMED_concept_name_34',
   'SNOMED_concept_class_id_4']].
 drop_duplicates().
 rename(
     columns={
         'SNOMED_concept_id_4' : 'SNOMED_concept_id',
         'SNOMED_concept_code_4' : 'SNOMED_concept_code',
         'SNOMED_concept_name_4' : 'SNOMED_concept_name',
         'SNOMED_concept_class_id_4' : 'SNOMED_concept_class_id'
     }
 ).
 set_index('MedDRA_concept_id_1').
 join(standard_reactions.
      drop_duplicates().
      set_index('MedDRA_concept_id')
     ).
 rename_axis('MedDRA_concept_id').
 reset_index().
 dropna(subset=['MedDRA_concept_id','SNOMED_concept_id','safetyreportid'])
)
standard_reactions_meddrasoc_to_snomed = \
(standard_reactions_meddrasoc_to_snomed.
 reindex(np.sort(standard_reactions_meddrasoc_to_snomed.columns),
         axis=1))
print(standard_reactions_meddrasoc_to_snomed.shape)
print(standard_reactions_meddrasoc_to_snomed.head())

(0, 10)
Empty DataFrame
Columns: [MedDRA_concept_class_id, MedDRA_concept_code, MedDRA_concept_id, MedDRA_concept_name, SNOMED_concept_class_id, SNOMED_concept_code, SNOMED_concept_id, SNOMED_concept_name_34, reaction_outcome, safetyreportid]
Index: []


/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [28]:
print(
    len(
        np.intersect1d(
            all_reports,
            standard_reactions_meddrasoc_to_snomed.safetyreportid.astype(str).unique()
        )
    )/len(all_reports)
)

0.0


In [29]:
del m_to_s_r
del df
del joinedpt
del joinedhlt
del joinedhlgt
del joinedsoc
del all_reports
del standard_reactions
del standard_reactions_meddrapt_to_snomed
del standard_reactions_meddrahlt_to_snomed
del standard_reactions_meddrahlgt_to_snomed
del standard_reactions_meddra_relationships